In [154]:
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import warnings
warnings.filterwarnings('ignore')

In [155]:
with open('Result files/Results.pickle', 'rb') as f:
    V=pickle.load(f)

f.close()

In [156]:
year_list=[2015,2020,2025,2030,2040,2050]
param=V["Parameters"]["RESOURCES_parameters"]
elec_price=param.loc[(["Electricity","Electricity_50%_LF","Electricity_25%_LF"]),"flow_cost_r"].reset_index().sort_values(by="YEAR")
elec_price=elec_price[(elec_price.SECTOR=="Steel") &(elec_price.YEAR<=2050)]
elec_price=elec_price[elec_price.YEAR.isin(year_list)].drop(["SECTOR","AREAS"],axis=1)
elec_price.set_index(["RESOURCES","YEAR"],inplace=True)
mini={}
maxi={}
start_year={"Electricity":2015,"Electricity_50%_LF":2025,"Electricity_25%_LF":2025}
for elec_type in ["Electricity","Electricity_50%_LF","Electricity_25%_LF"]:
    year_mini={}
    year_maxi={}
    for year in year_list:
        if year>=start_year[elec_type]:
            year_mini[year]=elec_price.loc[(elec_type,year),"flow_cost_r"].min()
            year_maxi[year]=elec_price.loc[(elec_type,year),"flow_cost_r"].max()
    mini[elec_type]=year_mini
    maxi[elec_type]=year_maxi
        
fig=go.Figure()
elec_type_color={"Electricity":"yellow","Electricity_50%_LF":"blue","Electricity_25%_LF":"red"}
for elec_type in ["Electricity","Electricity_50%_LF","Electricity_25%_LF"]:
    fig.add_trace(go.Scatter(x=list(maxi[elec_type].keys()),y=list(maxi[elec_type].values()),fill=None,mode="lines",line_color=elec_type_color[elec_type],showlegend=False))
    fig.add_trace(go.Scatter(x=list(mini[elec_type].keys()),y=list(mini[elec_type].values()),fill="tonexty",mode="lines",line_color=elec_type_color[elec_type],name=elec_type))
fig.update_layout(title='Electricity price distribution per load factor',title_x=0.5,#title_y=1,
#                    xaxis_title='Year',
                   yaxis_title='Price (€/MWh)')
fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=0.85,font = dict(size=20)
))

fig.update_layout(
    font=dict(
        family="Times New Roman",size=24)
    )
fig.update_layout( paper_bgcolor="rgba(255,255,255,255)", plot_bgcolor="rgba(255,255,255,255)")
fig.show()

In [157]:
param=V["Parameters"]["RESOURCES_parameters"]
price=param.loc[(["Biomass","Coal","Gas","Oil"],"Steel"),"flow_cost_r"].reset_index().sort_values(by="YEAR")
price=price[(price.YEAR<=2050)]
param=V["Parameters"]["TECHNOLOGIES_parameters"]
biogasprice=param.loc[('Biogas_Digester',"Steel"),'flow_cost_t'].reset_index().sort_values(by="YEAR")
biogasprice=biogasprice[(biogasprice.YEAR<=2050)].drop("AREAS",axis=1).set_index("YEAR")

param=V["Parameters"]["RESOURCES_parameters"]
biomassprice=param.loc[(["Biomass"],"Chemistry"),"flow_cost_r"].reset_index().sort_values(by="YEAR")
biomassprice=biomassprice[biomassprice.YEAR<=2050].drop(["AREAS","SECTOR","RESOURCES"],axis=1).set_index("YEAR")
elecprice=param.loc[(["Electricity"],"Chemistry"),"flow_cost_r"].reset_index().sort_values(by="YEAR")
elecprice=elecprice[elecprice.YEAR<=2050].drop(["AREAS","SECTOR","RESOURCES"],axis=1).set_index("YEAR")

carbon_tax=V["Parameters"]["SECTOR_parameters"]["carbon_tax"]

year_list=[2015,2020,2025,2030,2040,2050]

mini_biogas={}
maxi_biogas={}
mini_biomass={}
maxi_biomass={}
mini_elec={}
maxi_elec={}
for year in year_list:   
    mini_biogas[year]=biogasprice.loc[(year),"flow_cost_t"].min()/13.1
    maxi_biogas[year]=biogasprice.loc[(year),"flow_cost_t"].max()/13.1
    mini_biomass[year]=biomassprice.loc[(year),"flow_cost_r"].min()/7.89
    maxi_biomass[year]=biomassprice.loc[(year),"flow_cost_r"].max()/7.89
    mini_elec[year]=elecprice.loc[(year),"flow_cost_r"].min()
    maxi_elec[year]=elecprice.loc[(year),"flow_cost_r"].max()

        
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(go.Scatter(x=list(maxi_biogas.keys()),y=list(maxi_biogas.values()),fill=None,mode="lines",line_color="green",showlegend=False))
fig.add_trace(go.Scatter(x=list(mini_biogas.keys()),y=list(mini_biogas.values()),fill="tonexty",mode="lines",line_color="green",name="Biogas"))
fig.add_trace(go.Scatter(x=list(maxi_biomass.keys()),y=list(maxi_biomass.values()),fill=None,mode="lines",line_color="firebrick",showlegend=False))
fig.add_trace(go.Scatter(x=list(mini_biomass.keys()),y=list(mini_biomass.values()),fill="tonexty",mode="lines",line_color="firebrick",name="Biomass"))
fig.add_trace(go.Scatter(x=list(maxi_elec.keys()),y=list(maxi_elec.values()),fill=None,mode="lines",line_color="yellow",showlegend=False))
fig.add_trace(go.Scatter(x=list(mini_elec.keys()),y=list(mini_elec.values()),fill="tonexty",mode="lines",line_color="yellow",name="Electricity"))


fig.add_trace(go.Scatter(x=price[(price.AREAS=="France") & (price.RESOURCES=="Gas")].YEAR,
                             y=price[(price.AREAS=="France")& (price.RESOURCES=="Gas")].flow_cost_r/13.1,
                    mode='lines+markers',
                    name="Natural Gas",line=dict(color="purple", width=3),marker=dict(symbol='diamond'),))
fig.add_trace(go.Scatter(x=price[(price.AREAS=="France") & (price.RESOURCES=="Coal")].YEAR,
                             y=price[(price.AREAS=="France")& (price.RESOURCES=="Coal")].flow_cost_r/8.06,
                    mode='lines+markers',
                    name="Coal",line=dict(color="black", width=3),marker=dict(symbol='diamond'),))

fig.add_trace(go.Scatter(x=price[(price.AREAS=="France") & (price.RESOURCES=="Oil")].YEAR,
                             y=price[(price.AREAS=="France")& (price.RESOURCES=="Oil")].flow_cost_r/10.83,
                    mode='lines+markers',
                    name="Oil",line=dict(color="grey", width=3),marker=dict(symbol='diamond'),))

fig.add_trace(go.Scatter(x=list(range(2015,2051)),
                        y=carbon_tax.loc[("Steel","France",list(range(2015,2051)))].tolist(),
                         line=dict(color="red", width=3),name="Carbon tax",showlegend=False),secondary_y=True)

fig.update_layout(title='Energy resource prices evolution and carbon tax',title_x=0.5,#title_y=1,
#                    xaxis_title='Year',
                   yaxis_title='Price (€/MWh)')
fig.update_yaxes(title_text="Carbon tax (€/tCO<sub>2</sub>)",color="red", secondary_y=True)

fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=0.85,font = dict(size=20)
))

fig.update_layout(
    font=dict(
        family="Times New Roman",size=24)
    )
fig.update_layout( paper_bgcolor="rgba(255,255,255,255)", plot_bgcolor="rgba(255,255,255,255)")
fig.show()

In [158]:
year=2050
primary_resource_conversion={"Hydrogen":33.3,"Gas":13.1,"Coal":8.06,"Oil":10.89,"Biomass":7.89,"Electricity":1,"MeOH":5.54,"Waste":4.72,"Olefins":5.78,"Ammonia":5.17,"Naphtha":12.47}
primary_resource_color={"Hydrogen":"cyan","Gas":"purple","Coal":"black","Oil":"grey","Biomass":"brown","Electricity":"yellow","MeOH":"red","Waste":"green","Olefins":"darkcyan","Ammonia":"darksalmon","Naphtha":"wheat"}
sector_color={"Steel":"grey","Chemistry":"pink","Cement":"green","Glass":"orange","E-Fuels":"blue"}
captured_emissions_sector_color={"Steel":"black","Chemistry":"purple","Cement":"brown","Glass":"yellow","E-fuels":"cyan"}

resource_inflow=V["V_resource_inflow"].reset_index().rename(columns={"level_0":"Resource","level_1":"Sector","level_2":"Country","level_3":"Year",0:"Flow"}).sort_values(by="Year").groupby(by=["Resource","Sector","Country","Year"]).sum().reset_index()
u=resource_inflow.loc[resource_inflow.Resource.isin(["Electricity","Electricity_50%_LF","Electricity_25%_LF"])].drop("Resource",axis=1).groupby(by=["Sector","Country","Year"]).sum().reset_index()
u["Resource"]="Electricity"
resource_inflow=resource_inflow.loc[~resource_inflow.Resource.isin(["Electricity","Electricity_50%_LF","Electricity_25%_LF"])]
resource_inflow=pd.concat([resource_inflow,u],ignore_index=True)
resource_inflow=resource_inflow[resource_inflow.Year<=year]

resource_outflow=V["V_resource_outflow"].reset_index().rename(columns={"level_0":"Resource","level_1":"Sector","level_2":"Country","level_3":"Year",0:"Flow"}).sort_values(by="Year").groupby(by=["Resource","Sector","Country","Year"]).sum().reset_index()
u=resource_outflow.loc[resource_outflow.Resource.isin(["Electricity","Electricity_50%_LF","Electricity_25%_LF"])].drop("Resource",axis=1).groupby(by=["Sector","Country","Year"]).sum().reset_index()
u["Resource"]="Electricity"
resource_outflow=resource_outflow.loc[~resource_outflow.Resource.isin(["Electricity","Electricity_50%_LF","Electricity_25%_LF"])]
resource_outflow=pd.concat([resource_outflow,u],ignore_index=True)
resource_outflow=resource_outflow[resource_outflow.Year<=year]

resource_flow=V["V_resource_flow"].reset_index().rename(columns={"level_0":"Resource","level_1":"Sector","level_2":"Country","level_3":"Year",0:"Flow"}).sort_values(by="Year").groupby(by=["Resource","Sector","Country","Year"]).sum().reset_index()
u=resource_flow.loc[resource_flow.Resource.isin(["Electricity","Electricity_50%_LF","Electricity_25%_LF"])].drop("Resource",axis=1).groupby(by=["Sector","Country","Year"]).sum().reset_index()
u["Resource"]="Electricity"
resource_flow=resource_flow.loc[~resource_flow.Resource.isin(["Electricity","Electricity_50%_LF","Electricity_25%_LF"])]
resource_flow=pd.concat([resource_flow,u],ignore_index=True)
resource_flow=resource_flow[resource_flow.Year<=year]

resource_tech_inflow=V["V_resource_tech_type_inflow"].reset_index().rename(columns={"level_0":"Technology","level_1":"Tech_type","level_3":"Resource","level_2":"Sector","level_4":"Country","level_5":"Year",0:"Flow"}).sort_values(by="Year").groupby(by=["Technology","Tech_type","Resource","Sector","Country","Year"]).sum().reset_index()
u=resource_tech_inflow.loc[resource_tech_inflow.Resource.isin(["Electricity","Electricity_50%_LF","Electricity_25%_LF"])].drop("Resource",axis=1).groupby(by=["Technology","Tech_type","Sector","Country","Year"]).sum().reset_index()
u["Resource"]="Electricity"
resource_tech_inflow=resource_tech_inflow.loc[~resource_tech_inflow.Resource.isin(["Electricity","Electricity_50%_LF","Electricity_25%_LF"])]
resource_tech_inflow=pd.concat([resource_tech_inflow,u],ignore_index=True)
resource_tech_inflow=resource_tech_inflow[resource_tech_inflow.Year<=year]

resource_tech_outflow=V["V_resource_tech_type_outflow"].reset_index().rename(columns={"level_0":"Technology","level_1":"Tech_type","level_3":"Resource","level_2":"Sector","level_4":"Country","level_5":"Year",0:"Flow"}).sort_values(by="Year").groupby(by=["Technology","Tech_type","Resource","Sector","Country","Year"]).sum().reset_index()
u=resource_tech_outflow.loc[resource_tech_outflow.Resource.isin(["Electricity","Electricity_50%_LF","Electricity_25%_LF"])].drop("Resource",axis=1).groupby(by=["Technology","Tech_type","Sector","Country","Year"]).sum().reset_index()
u["Resource"]="Electricity"
resource_tech_outflow=resource_tech_outflow.loc[~resource_tech_outflow.Resource.isin(["Electricity","Electricity_50%_LF","Electricity_25%_LF"])]
resource_tech_outflow=pd.concat([resource_tech_outflow,u],ignore_index=True)
resource_tech_outflow=resource_tech_outflow[resource_tech_outflow.Year<=year]


resource_imports=V["V_resource_imports"].reset_index().rename(columns={"level_0":"Resource","level_1":"Sector","level_2":"Country","level_3":"Year",0:"Flow"}).sort_values(by="Year").groupby(by=["Resource","Sector","Country","Year"]).sum().reset_index()
u=resource_imports.loc[resource_imports.Resource.isin(["Electricity","Electricity_50%_LF","Electricity_25%_LF"])].drop("Resource",axis=1).groupby(by=["Sector","Country","Year"]).sum().reset_index()
u["Resource"]="Electricity"
resource_imports=resource_imports.loc[~resource_imports.Resource.isin(["Electricity","Electricity_50%_LF","Electricity_25%_LF"])]
resource_imports=pd.concat([resource_imports,u],ignore_index=True)
resource_imports=resource_imports[resource_imports.Year<=year]


resource_exports=V["V_resource_exports"].reset_index().rename(columns={"level_0":"Resource","level_1":"Sector","level_2":"Country","level_3":"Year",0:"Flow"}).sort_values(by="Year").groupby(by=["Resource","Sector","Country","Year"]).sum().reset_index()
u=resource_exports.loc[resource_exports.Resource.isin(["Electricity","Electricity_50%_LF","Electricity_25%_LF"])].drop("Resource",axis=1).groupby(by=["Sector","Country","Year"]).sum().reset_index()
u["Resource"]="Electricity"
resource_exports=resource_exports.loc[~resource_exports.Resource.isin(["Electricity","Electricity_50%_LF","Electricity_25%_LF"])]
resource_exports=pd.concat([resource_exports,u],ignore_index=True)
resource_exports=resource_exports[resource_exports.Year<=year]

tech_usage_flow=resource_tech_outflow.copy()

elec_consum_fc=V["V_resource_flow"].reset_index().rename(columns={"level_0":"Resource","level_1":"Sector","level_2":"Country","level_3":"Year",0:"Flow"}).sort_values(by="Year").groupby(by=["Resource","Sector","Country","Year"]).sum().reset_index()
elec_consum_fc=elec_consum_fc[elec_consum_fc.Resource.isin(["Electricity","Electricity_50%_LF","Electricity_25%_LF"])]
elec_consum_fc=elec_consum_fc[elec_consum_fc.Year<=year]

for resource in primary_resource_conversion.keys():
    resource_inflow.loc[resource_inflow.Resource==resource,"Flow"]=primary_resource_conversion[resource]*resource_inflow.loc[resource_inflow.Resource==resource,"Flow"]*1e-6
    resource_outflow.loc[resource_outflow.Resource==resource,"Flow"]=primary_resource_conversion[resource]*resource_outflow.loc[resource_outflow.Resource==resource,"Flow"]*-1e-6
    resource_flow.loc[resource_flow.Resource==resource,"Flow"]=primary_resource_conversion[resource]*resource_flow.loc[resource_flow.Resource==resource,"Flow"]*1e-6
    resource_tech_inflow.loc[resource_tech_inflow.Resource==resource,"Flow"]=primary_resource_conversion[resource]*resource_tech_inflow.loc[resource_tech_inflow.Resource==resource,"Flow"]*1e-6
    resource_tech_outflow.loc[resource_tech_outflow.Resource==resource,"Flow"]=primary_resource_conversion[resource]*resource_tech_outflow.loc[resource_tech_outflow.Resource==resource,"Flow"]*-1e-6
    resource_imports.loc[resource_imports.Resource==resource,"Flow"]=primary_resource_conversion[resource]*resource_imports.loc[resource_imports.Resource==resource,"Flow"]*1e-6
    resource_exports.loc[resource_exports.Resource==resource,"Flow"]=primary_resource_conversion[resource]*resource_exports.loc[resource_exports.Resource==resource,"Flow"]*-1e-6


emissions=V["V_emissions"].reset_index().rename(columns={"level_0":"Sector","level_1":"Country","level_2":"Year",0:"Emissions"}).sort_values(by="Year")
emissions=emissions[emissions.Year<=year]
captured_emissions=V["V_captured_emissions"].reset_index().rename(columns={"level_0":"Technology","level_1":"Sector","level_2":"Country","level_3":"Year",0:"Captured_emissions"}).sort_values(by="Year")
captured_emissions=captured_emissions[captured_emissions.Year<=year]

tech_usage_coef=V["V_technology_use_coef"].reset_index().rename(columns={"level_0":"Technology","level_1":"Tech_type","level_2":"Sector","level_3":"Country","level_4":"Year",0:"Coef"}).sort_values(by="Year")
tech_usage_coef=tech_usage_coef[tech_usage_coef.Year<=year]
tech_usage_capacity_coef=V["V_technology_use_coef_capacity"].reset_index().rename(columns={"level_0":"Technology","level_1":"Sector","level_2":"Country","level_3":"Year",0:"Coef"}).sort_values(by="Year")
tech_usage_capacity_coef=tech_usage_capacity_coef[tech_usage_capacity_coef.Year<=year]

tech_capacity=V["V_resource_tech_type_capacity"].reset_index().rename(columns={"level_0":"Technology","level_1":"Tech_type","level_3":"Resource","level_2":"Sector","level_4":"Country","level_5":"Year",0:"Flow_capa"}).sort_values(by="Year")
tech_capacity=tech_capacity[tech_capacity.Year<=year]
added_capa_coef=V["V_added_capacity"].reset_index().rename(columns={"level_0":"Technology","level_1":"Sector","level_2":"Country","level_3":"Year",0:"Coef_capa"}).sort_values(by="Year")
added_capa_coef=added_capa_coef[added_capa_coef.Year<=year]
remove_capa_coef=V["V_removed_capacity"].reset_index().rename(columns={"level_0":"Technology","level_1":"Sector","level_2":"Country","level_3":"Year",0:"Coef_capa"}).sort_values(by="Year")
remove_capa_coef=remove_capa_coef[remove_capa_coef.Year<=year]

sector_cost=V["V_cost"].reset_index().rename(columns={"level_0":"Sector","level_1":"Country","level_2":"Year",0:"Cost"}).sort_values(by="Year")
sector_cost=sector_cost[sector_cost.Year<=year]
tech_cost=V["V_technology_cost"].reset_index().rename(columns={"level_0":"Technology","level_1":"Sector","level_2":"Country","level_3":"Year",0:"Cost"}).sort_values(by="Year")
tech_cost=tech_cost[tech_cost.Year<=year]
carbon_cost=V["V_carbon_cost"].reset_index().rename(columns={"level_0":"Sector","level_1":"Country","level_2":"Year",0:"Cost"}).sort_values(by="Year")
carbon_cost=carbon_cost[carbon_cost.Year<=year]
ccs_capex_cost=V["V_ccs_capex_cost"].reset_index().rename(columns={"level_0":"Sector","level_1":"Country","level_2":"Year",0:"Cost"}).sort_values(by="Year")
ccs_capex_cost=ccs_capex_cost[ccs_capex_cost.Year<=year]
ccs_opex_cost=V["V_ccs_opex_cost"].reset_index().rename(columns={"level_0":"Sector","level_1":"Country","level_2":"Year",0:"Cost"}).sort_values(by="Year")
ccs_opex_cost=ccs_opex_cost[ccs_opex_cost.Year<=year]




In [159]:

fig =go.Figure()
emissions_agg=emissions.drop(["Sector","Country"],axis=1).groupby(by=["Year"]).sum().reset_index()
captured_emissions_agg=captured_emissions.drop(["Technology","Sector","Country"],axis=1).groupby(by="Year").sum().reset_index()

fig.add_trace(go.Scatter(
    name="Emissions",
    x=emissions_agg.Year, y=emissions_agg.Emissions*1e-6,
    mode='lines',
    line=dict(width=0, color='grey'),
    stackgroup='one'
))
fig.add_trace(go.Scatter(
    name="Captured emissions",
    x=captured_emissions_agg.Year, y=captured_emissions_agg.Captured_emissions*1e-6,
    mode='lines',
    line=dict(width=0, color='black'),
    stackgroup='one'
))

em_ini=emissions_agg[emissions_agg.Year==2015]["Emissions"].values[0]*1e-6
fig.add_trace(go.Scatter(
    name="SNBC",
    x=[2030,2050], y=[em_ini*0.65,em_ini*0.2],
    mode='lines',
    line=dict(width=3, color='black')))

# fig.add_trace(go.Scatter(
#     name="Forced trajectory",
#     x=[2035,2040,2050], y=[em_ini*0.5,em_ini*0.3,em_ini*0.05],
#     mode='markers+lines',
#     marker = dict(
#           color = 'red',
#           size = 0,line=dict(
#             color = 'red',
#             width = 2))))

#     fig.update_xaxes(title_text="Year", row=1, col=i+1)

fig.update_layout(title='CO<sub>2</sub> emissions in EU-5+1',title_x=0.5,title_y=0.97,
                   yaxis_title='Emissions (MtCO<sub>2</sub>)')
# fig.update_yaxes(range=[0,330])
fig.update_layout(
    font=dict(
        family="Times New Roman",size=24)
    )
fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.06,
    xanchor="right",
    x=0.9
))
fig.update_annotations(font_size=24)
# fig.add_annotation(dict(font=dict(color="black",size=28),
#                                         x=-0.05,
#                                         y=1.15,
#                                         showarrow=False,
#                                         text="(a)",
#                                         textangle=0,
#                                         xanchor='left',
#                                         xref="paper",
#                                         yref="paper"))
fig.update_layout( paper_bgcolor="rgba(0,0,0,0)", plot_bgcolor="rgba(0,0,0,0)")
fig.show()

In [160]:

fig =go.Figure()
emissions_agg=emissions.drop(["Country"],axis=1).groupby(by=["Sector","Year"]).sum().reset_index()

for sector in sector_color.keys():
    fig.add_trace(go.Scatter(
        name=sector,
        x=emissions_agg[(emissions_agg.Sector==sector)&(emissions_agg.Emissions>=0)].Year,
        y=emissions_agg[(emissions_agg.Sector==sector)&(emissions_agg.Emissions>=0)].Emissions*1e-6,
        mode='lines',
        line=dict(width=0, color=sector_color[sector]),
        stackgroup='one'
    ))
    fig.add_trace(go.Scatter(
        name=sector,
        x=emissions_agg[(emissions_agg.Sector==sector)&(emissions_agg.Emissions<=0)].Year,
        y=emissions_agg[(emissions_agg.Sector==sector)&(emissions_agg.Emissions<=0)].Emissions*1e-6,
        mode='lines',
        line=dict(width=0, color=sector_color[sector]),
        stackgroup='two',showlegend=False
    ))
    



fig.update_layout(title='CO<sub>2</sub> emissions per sector in EU-5+1',title_x=0.5,title_y=0.97,
                   yaxis_title='Emissions (MtCO<sub>2</sub>)')
# fig.update_yaxes(range=[0,350])
fig.update_layout(
    font=dict(
        family="Times New Roman",size=24)
    )
fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.0,
    xanchor="right",
    x=0.75
))
fig.update_annotations(font_size=24)
# fig.add_annotation(dict(font=dict(color="black",size=28),
#                                         x=-0.05,
#                                         y=1.15,
#                                         showarrow=False,
#                                         text="(b)",
#                                         textangle=0,
#                                         xanchor='left',
#                                         xref="paper",
#                                         yref="paper"))
fig.update_layout( paper_bgcolor="rgba(0,0,0,0)", plot_bgcolor="rgba(0,0,0,0)")
fig.show()

In [161]:
energy_input=resource_inflow[(resource_inflow.Resource.isin(primary_resource_conversion.keys()))]
# energy_input=energy_input[energy_input.Sector=="E-Fuels"]
energy_output=resource_outflow[(resource_outflow.Resource.isin(primary_resource_conversion.keys()))]
# energy_output=energy_output[energy_output.Sector=="E-Fuels"]

energy_input=energy_input.drop(["Sector","Country"],axis=1).groupby(by=["Resource","Year"]).sum().reset_index()
energy_output=energy_output.drop(["Sector","Country"],axis=1).groupby(by=["Resource","Year"]).sum().reset_index()
df_merged=pd.concat([energy_input,energy_output],ignore_index=True)
fig = go.Figure()

for resource in primary_resource_conversion.keys():
    if resource not in ["Olefins","Ammonia","Naphtha","CO"]:
        fig.add_trace(go.Scatter(x=energy_input[energy_input.Resource==resource].Year,y=energy_input[energy_input.Resource==resource].Flow,name=resource,
                         line=dict(width=0,color=primary_resource_color[resource]),
                             fillcolor=primary_resource_color[resource],mode='lines',stackgroup='one'))
        fig.add_trace(go.Scatter(x=energy_output[energy_output.Resource==resource].Year,y=energy_output[energy_output.Resource==resource].Flow,name=resource,
                         line=dict(width=0,color=primary_resource_color[resource]),
                             fillcolor=primary_resource_color[resource],mode='lines',stackgroup='two',showlegend=False))
fig.update_layout(title='Energy flows in EU-5+1',title_x=0.5,title_y=0.97,
                   yaxis_title='Energy flows (TWh)')
fig.update_layout(
    font=dict(
        family="Times New Roman",size=24)
    )
fig.update_annotations(font_size=24)
fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.05,
    xanchor="right",
    x=1,font = dict(size=20)
))

# fig.add_annotation(dict(font=dict(color="black",size=28),
#                                         x=-0.05,
#                                         y=1.15,
#                                         showarrow=False,
#                                         text="(c)",
#                                         textangle=0,
#                                         xanchor='left',
#                                         xref="paper",
#                                         yref="paper"))

fig.update_layout( paper_bgcolor="rgba(0,0,0,0)", plot_bgcolor="rgba(0,0,0,0)")
fig.show()

In [162]:
gas_techs=["Biogas_Digester","Gasification","Gasification_noWGR","Naphta_cracking","Naphta_cracking_electrified"]


tech_name={"Electrolyser":"Electrolyser","SMR":"SMR","SMR_noWGR":"SMR","SMR_eboiler_noWGR":"SMR",
                       "eSMR":"SMR","eSMR_noWGR":"SMR","eSMR_eboiler_noWGR":"SMR",
                "Gasification":"Gasification","Gasification_noWGR":"Gasification","Pyrolysis":"Pyrolysis",
           "Naphta_cracking":"Naphta cracking","Naphta_cracking_electrified":"Naphta cracking"}
tech_name_list={"Biogas":["Biogas_Digester","Gasification","Gasification_noWGR"],
           "Gas by-product":["Naphta_cracking","Naphta_cracking_electrified"]}
gas_techs_name_colors={"Biogas":"green",
                       "Gas by-product":"darkkhaki"}





fig =go.Figure()
tech_usage=tech_usage_flow[(tech_usage_flow.Resource.isin(["Gas"]))]


energy_input=resource_imports[(resource_imports.Resource.isin(primary_resource_conversion.keys()))]
energy_input=energy_input.drop(["Sector","Country"],axis=1).groupby(by=["Resource","Year"]).sum().reset_index()

gas_tech_usage=tech_usage[tech_usage.Technology.isin(gas_techs)]
for tech in tech_name_list.keys():
    u=tech_usage[tech_usage.Technology.isin(tech_name_list[tech])]
    u=u[["Year","Flow"]].groupby(by=["Year"]).sum().reset_index()
    fig.add_trace(go.Scatter(x=u.Year,y=u.Flow*1e-6*13.1,name=tech,line=dict(width=0,color=gas_techs_name_colors[tech]),
                             fillcolor=gas_techs_name_colors[tech],hoverinfo="x+y",mode='lines',stackgroup='one'))

fig.add_trace(go.Scatter(x=energy_input[energy_input.Resource=="Gas"].Year,y=energy_input[energy_input.Resource=="Gas"].Flow,name="Natural Gas",
                         line=dict(width=0,color="purple"),
                             fillcolor="purple",mode='lines',stackgroup='one'))
#     fig.update_xaxes(title_text="Year", row=1, col=i+1)

fig.update_layout(title='Gas consumption in EU-5+1',  title_x=0.5,title_y=0.97,
                   yaxis_title='Consumption (TWh)')
fig.update_layout(
    font=dict(
        family="Times New Roman",size=24)
    )
fig.update_annotations(font_size=20)
fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.06,
    xanchor="right",
    x=0.75,font = dict(size=20)
))
# fig.update_yaxes(range=[0,5.5])
# fig.add_annotation(dict(font=dict(color="black",size=28),
#                                         x=-0.05,
#                                         y=1.15,
#                                         showarrow=False,
#                                         text="(e)",
#                                         textangle=0,
#                                         xanchor='left',
#                                         xref="paper",
#                                         yref="paper"))
fig.update_layout( paper_bgcolor="rgba(0,0,0,0)", plot_bgcolor="rgba(0,0,0,0)")
fig.show()


In [163]:
gas_techs=["Biogas_Digester"]


tech_name={"Electrolyser":"Electrolyser","SMR":"SMR","SMR_noWGR":"SMR","SMR_eboiler_noWGR":"SMR",
                       "eSMR":"SMR","eSMR_noWGR":"SMR","eSMR_eboiler_noWGR":"SMR",
                "Gasification":"Gasification","Gasification_noWGR":"Gasification","Pyrolysis":"Pyrolysis",
           "Naphta_cracking":"Naphta cracking","Naphta_cracking_electrified":"Naphta cracking"}
tech_name_list={"Biogas":["Biogas_Digester"]}
gas_techs_name_colors={"Biogas":"yellowgreen",}



fig =go.Figure()
#Biogas and other imports
tech_usage=tech_usage_flow[(tech_usage_flow.Resource.isin(["Gas"]))]
# tech_usage=tech_usage[tech_usage.Sector=="Glass"]

energy_input=resource_imports[(resource_imports.Resource.isin(primary_resource_conversion.keys()))]
# energy_input=energy_input[energy_input.Sector=="Glass"]
energy_input=energy_input.drop(["Sector","Country"],axis=1).groupby(by=["Resource","Year"]).sum().reset_index()

gas_tech_usage=tech_usage[tech_usage.Technology.isin(gas_techs)]
# gas_tech_usage=gas_tech_usage[gas_tech_usage.Sector=="Glass"]
for tech in tech_name_list.keys():
    u=tech_usage[tech_usage.Technology.isin(tech_name_list[tech])]
    u=u[["Year","Flow"]].groupby(by=["Year"]).sum().reset_index()
    fig.add_trace(go.Scatter(x=u.Year,y=u.Flow*1e-6*13.1,name=tech,line=dict(width=0,color=gas_techs_name_colors[tech]),
                             fillcolor=gas_techs_name_colors[tech],hoverinfo="x+y",mode='lines',stackgroup='one'))

fig.add_trace(go.Scatter(x=energy_input[energy_input.Resource=="Gas"].Year,y=energy_input[energy_input.Resource=="Gas"].Flow,name="Natural Gas",
                         line=dict(width=0,color="purple"),
                             fillcolor="purple",mode='lines',stackgroup='one'))
fig.add_trace(go.Scatter(x=energy_input[energy_input.Resource=="Hydrogen"].Year,y=energy_input[energy_input.Resource=="Hydrogen"].Flow,
                         name="Imported hydrogen",
                         line=dict(width=0,color="cyan"),
                             fillcolor="cyan",mode='lines',stackgroup='one'))
fig.add_trace(go.Scatter(x=energy_input[energy_input.Resource=="MeOH"].Year,y=energy_input[energy_input.Resource=="MeOH"].Flow,
                         name="Imported MeOH",
                         line=dict(width=0,color="red"),
                             fillcolor="red",mode='lines',stackgroup='one'))


    ##Consumption
    

energy_input=resource_inflow[(resource_inflow.Resource.isin(primary_resource_conversion.keys()))]
# energy_input=energy_input[energy_input.Sector=="Glass"]
energy_input=energy_input.drop(["Sector","Country"],axis=1).groupby(by=["Resource","Year"]).sum().reset_index()

for resource in primary_resource_conversion.keys():
    if resource not in ["Olefins","Ammonia","Naphtha","CO","Gas","Hydrogen","MeOH"]:
        fig.add_trace(go.Scatter(x=energy_input[energy_input.Resource==resource].Year,y=energy_input[energy_input.Resource==resource].Flow,name=resource,
                         line=dict(width=0,color=primary_resource_color[resource]),
                             fillcolor=primary_resource_color[resource],mode='lines',stackgroup='one'))



fig.update_layout(title='Final energy consumption in EU-5+1',  title_x=0.5,title_y=1,
                   yaxis_title='Consumption (TWh)')
fig.update_layout(
    font=dict(
        family="Times New Roman",size=22)
    )
fig.update_annotations(font_size=20)
fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.06,
    xanchor="left",
    x=-0.005,font = dict(size=16)
))
# fig.update_yaxes(range=[0,1250])
# fig.add_annotation(dict(font=dict(color="black",size=28),
#                                         x=-0.05,
#                                         y=1.15,
#                                         showarrow=False,
#                                         text="(e)",
#                                         textangle=0,
#                                         xanchor='left',
#                                         xref="paper",
#                                         yref="paper"))
fig.update_layout( paper_bgcolor="rgba(0,0,0,0)", plot_bgcolor="rgba(0,0,0,0)")
fig.show()


In [164]:
gas_techs=["Biogas_Digester"]


tech_name={"Electrolyser":"Electrolyser","SMR":"SMR","SMR_noWGR":"SMR","SMR_eboiler_noWGR":"SMR",
                       "eSMR":"SMR","eSMR_noWGR":"SMR","eSMR_eboiler_noWGR":"SMR",
                "Gasification":"Gasification","Gasification_noWGR":"Gasification","Pyrolysis":"Pyrolysis",
           "Naphta_cracking":"Naphta cracking","Naphta_cracking_electrified":"Naphta cracking"}
tech_name_list={"Biogas":["Biogas_Digester"]}
gas_techs_name_colors={"Biogas":"yellowgreen",}


fig =go.Figure()
df_merged=pd.DataFrame(columns=["Year","Flow","Type"])
#Biogas and other imports
tech_usage=tech_usage_flow[(tech_usage_flow.Resource.isin(["Gas"]))]

energy_input=resource_imports[(resource_imports.Resource.isin(primary_resource_conversion.keys()))]
energy_input=energy_input.drop(["Sector","Country"],axis=1).groupby(by=["Resource","Year"]).sum().reset_index()

gas_tech_usage=tech_usage[tech_usage.Technology.isin(gas_techs)]
biogas=gas_tech_usage[["Year","Flow"]].groupby("Year").sum().reset_index()
biogas["Type"]="Domestic"
biogas["Flow"]=biogas["Flow"]*1e-6
#     print(df_merged)
df_merged=pd.concat([df_merged,biogas],ignore_index=True)
#     print(df_merged)
h2meohimports=energy_input[energy_input.Resource.isin(["Hydrogen","MeOH"])][["Year","Flow"]].groupby("Year").sum().reset_index()
h2meohimports["Type"]="Imported"
#     print(h2meohimports)
df_merged=pd.concat([df_merged,h2meohimports],ignore_index=True)
#     print(df_merged)

natgas=energy_input[energy_input.Resource.isin(["Gas"])][["Year","Flow"]].groupby("Year").sum().reset_index()
natgas["Type"]="Imported"
df_merged=pd.concat([df_merged,natgas],ignore_index=True)

##Consumption

energy_input=resource_inflow[(resource_inflow.Resource.isin(primary_resource_conversion.keys()))]
energy_input=energy_input.drop(["Sector","Country"],axis=1).groupby(by=["Resource","Year"]).sum().reset_index()

lowc=energy_input[energy_input.Resource.isin(["Electricity","Biomass","Waste"])][["Year","Flow"]].groupby("Year").sum().reset_index()
lowc["Type"]="Domestic"
df_merged=pd.concat([df_merged,lowc],ignore_index=True)

highc=energy_input[energy_input.Resource.isin(["Coal","Oil"])][["Year","Flow"]].groupby("Year").sum().reset_index()
highc["Type"]="Imported"
df_merged=pd.concat([df_merged,highc],ignore_index=True)
#     print(df_merged)
df_merged=df_merged.groupby(["Year","Type"]).sum().reset_index()
#     print(df_merged)
colors={"Imported":"#005cc6","Domestic":"#f8b740"}
for t in list(colors.keys()):
    fig.add_trace(go.Scatter(x=df_merged[df_merged.Type==t].Year,y=df_merged[df_merged.Type==t].Flow,name=t,
                         line=dict(width=0,color=colors[t]),
                             fillcolor=colors[t],mode='lines',stackgroup='one'))
        
fig.update_layout(title='Energy consumption origin in EU-5+1',  title_x=0.5,title_y=0.97,
                   yaxis_title='Energy distribution')
fig.update_layout(
    font=dict(
        family="Times New Roman",size=24)
    )
fig.update_annotations(font_size=20)
fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.05,
    xanchor="right",
    x=0.6,font = dict(size=20)
))
# fig.update_yaxes(range=[0,5.5])
# fig.add_annotation(dict(font=dict(color="black",size=28),
#                                         x=-0.05,
#                                         y=1.15,
#                                         showarrow=False,
#                                         text="(e)",
#                                         textangle=0,
#                                         xanchor='left',
#                                         xref="paper",
#                                         yref="paper"))
fig.update_layout( paper_bgcolor="rgba(0,0,0,0)", plot_bgcolor="rgba(0,0,0,0)")
fig.update_traces(groupnorm="percent")
fig.update_layout(yaxis=dict(ticksuffix="%"),yaxis2=dict(ticksuffix="%"),yaxis3=dict(ticksuffix="%")
                 )
fig.show()


In [165]:
gas_techs=["Biogas_Digester"]


tech_name={"Electrolyser":"Electrolyser","SMR":"SMR","SMR_noWGR":"SMR","SMR_eboiler_noWGR":"SMR",
                       "eSMR":"SMR","eSMR_noWGR":"SMR","eSMR_eboiler_noWGR":"SMR",
                "Gasification":"Gasification","Gasification_noWGR":"Gasification","Pyrolysis":"Pyrolysis",
           "Naphta_cracking":"Naphta cracking","Naphta_cracking_electrified":"Naphta cracking"}
tech_name_list={"Biogas":["Biogas_Digester"]}
gas_techs_name_colors={"Biogas":"yellowgreen",}



fig =go.Figure()
df_merged=pd.DataFrame(columns=["Year","Flow","Type"])
#Biogas and other imports
tech_usage=tech_usage_flow[(tech_usage_flow.Resource.isin(["Gas"]))]


energy_input=resource_imports[(resource_imports.Resource.isin(primary_resource_conversion.keys()))]
energy_input=energy_input.drop(["Sector","Country"],axis=1).groupby(by=["Resource","Year"]).sum().reset_index()

gas_tech_usage=tech_usage[tech_usage.Technology.isin(gas_techs)]
biogas=gas_tech_usage[["Year","Flow"]].groupby("Year").sum().reset_index()
biogas["Type"]="Low carbon"
biogas["Flow"]=biogas["Flow"]*1e-6
#     print(df_merged)
df_merged=pd.concat([df_merged,biogas],ignore_index=True)
#     print(df_merged)
h2meohimports=energy_input[energy_input.Resource.isin(["Hydrogen","MeOH"])][["Year","Flow"]].groupby("Year").sum().reset_index()
h2meohimports["Type"]="Depends"
#     print(h2meohimports)
df_merged=pd.concat([df_merged,h2meohimports],ignore_index=True)
#     print(df_merged)

natgas=energy_input[energy_input.Resource.isin(["Gas"])][["Year","Flow"]].groupby("Year").sum().reset_index()
natgas["Type"]="High carbon"
df_merged=pd.concat([df_merged,natgas],ignore_index=True)

##Consumption

energy_input=resource_inflow[(resource_inflow.Resource.isin(primary_resource_conversion.keys()))]
energy_input=energy_input.drop(["Sector","Country"],axis=1).groupby(by=["Resource","Year"]).sum().reset_index()

lowc=energy_input[energy_input.Resource.isin(["Electricity","Biomass","Waste"])][["Year","Flow"]].groupby("Year").sum().reset_index()
lowc["Type"]="Low carbon"
df_merged=pd.concat([df_merged,lowc],ignore_index=True)

highc=energy_input[energy_input.Resource.isin(["Coal","Oil"])][["Year","Flow"]].groupby("Year").sum().reset_index()
highc["Type"]="High carbon"
df_merged=pd.concat([df_merged,highc],ignore_index=True)
#     print(df_merged)
df_merged=df_merged.groupby(["Year","Type"]).sum().reset_index()
#     print(df_merged)
colors={"Low carbon":"green","Depends":"grey","High carbon":"black"}
for t in list(colors.keys()):
    fig.add_trace(go.Scatter(x=df_merged[df_merged.Type==t].Year,y=df_merged[df_merged.Type==t].Flow,name=t,
                         line=dict(width=0,color=colors[t]),
                             fillcolor=colors[t],mode='lines',stackgroup='one'))

fig.update_layout(title='Primary energy consumption carbon distribution in EU-5+1',  title_x=0.5,title_y=0.97,
                   yaxis_title='Energy distribution')
fig.update_layout(
    font=dict(
        family="Times New Roman",size=24)
    )
fig.update_annotations(font_size=20)
fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.05,
    xanchor="right",
    x=0.75,font = dict(size=20)
))
# fig.update_yaxes(range=[0,5.5])
# fig.add_annotation(dict(font=dict(color="black",size=28),
#                                         x=-0.05,
#                                         y=1.15,
#                                         showarrow=False,
#                                         text="(e)",
#                                         textangle=0,
#                                         xanchor='left',
#                                         xref="paper",
#                                         yref="paper"))
fig.update_layout( paper_bgcolor="rgba(0,0,0,0)", plot_bgcolor="rgba(0,0,0,0)")
fig.update_traces(groupnorm="percent")
fig.update_layout(yaxis=dict(ticksuffix="%"),yaxis2=dict(ticksuffix="%"),yaxis3=dict(ticksuffix="%")
                 )
fig.show()


In [166]:
fig =go.Figure()
r_flow=resource_flow.drop(["Sector","Country"],axis=1).groupby(by=["Resource","Year"]).sum().reset_index()
#     print(resource_flow[(resource_flow.Year==2045)&(resource_flow.Resource=="Gas")])
for resource in primary_resource_conversion.keys():
    if resource not in ["Olefins","Ammonia","Naphtha","CO","Electricity","Biomass","Waste"]:
        fig.add_trace(go.Scatter(x=r_flow[(r_flow.Resource==resource) & (r_flow.Flow>0)].Year,y=r_flow[(r_flow.Resource==resource) & (r_flow.Flow>0)].Flow,name=resource,
                             line=dict(width=0,color=primary_resource_color[resource]),
                             fillcolor=primary_resource_color[resource],hoverinfo="x+y",mode='lines',stackgroup='one'))

#     fig.update_xaxes(title_text="Year", row=1, col=i+1)
    
fig.update_layout(title='Energy imports in EU-5+1', title_x=0.5,title_y=0.97,
                   yaxis_title='Energy imports (TWh)')
fig.update_layout(
    font=dict(
        family="Times New Roman",size=24)
    )
fig.update_annotations(font_size=24)
fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.06,
    xanchor="right",
    x=0.75,font = dict(size=20)
))
# fig.add_annotation(dict(font=dict(color="black",size=28),
#                                         x=-0.05,
#                                         y=1.15,
#                                         showarrow=False,
#                                         text="(d)",
#                                         textangle=0,
#                                         xanchor='left',
#                                         xref="paper",
#                                         yref="paper"))

fig.update_layout( paper_bgcolor="rgba(0,0,0,0)", plot_bgcolor="rgba(0,0,0,0)")
fig.show()

In [167]:

fig =go.Figure()
r_flow=resource_imports.drop(["Sector","Country"],axis=1).groupby(by=["Resource","Year"]).sum().reset_index()
#     print(resource_flow[(resource_flow.Year==2045)&(resource_flow.Resource=="Gas")])
for resource in primary_resource_conversion.keys():
    if resource in ["Coal","Oil","Gas"]:
        fig.add_trace(go.Scatter(x=r_flow[(r_flow.Resource==resource) & (r_flow.Flow>=0)].Year,y=r_flow[(r_flow.Resource==resource) & (r_flow.Flow>=0)].Flow,name=resource,
                             line=dict(width=0,color=primary_resource_color[resource]),
                             fillcolor=primary_resource_color[resource],hoverinfo="x+y",mode='lines',stackgroup='one'))

#     fig.update_xaxes(title_text="Year", row=1, col=i+1)
    
fig.update_layout(title='Fossil energy imports in EU-5+1', title_x=0.5,title_y=0.97,
                   yaxis_title='Fossil imports (TWh)')
fig.update_layout(
    font=dict(
        family="Times New Roman",size=24)
    )
fig.update_annotations(font_size=24)
fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.06,
    xanchor="right",
    x=0.65,font = dict(size=20)
))
# fig.add_annotation(dict(font=dict(color="black",size=28),
#                                         x=-0.05,
#                                         y=1.15,
#                                         showarrow=False,
#                                         text="(d)",
#                                         textangle=0,
#                                         xanchor='left',
#                                         xref="paper",
#                                         yref="paper"))

fig.update_layout( paper_bgcolor="rgba(0,0,0,0)", plot_bgcolor="rgba(0,0,0,0)")
fig.show()

In [168]:

hydrogen_techs=["Electrolyser","SMR",
                       "eSMR",
                "Gasification","Pyrolysis","Naphta_cracking","Naphta_cracking_electrified"]
hydrogen_techs_colors={"Electrolyser":"green","SMR":"black","SMR_noWGR":"black","SMR_eboiler_noWGR":"grey",
                       "eSMR":"yellow","eSMR_noWGR":"orange","eSMR_eboiler_noWGR":"orange",
                "Gasification":"firebrick","Gasification_noWGR":"firebrick","Pyrolysis":"turquoise",
                       "Naphta_cracking":"darkkhaki","Naphta_cracking_electrified":"darkkhaki"}

tech_name={"Electrolyser":"Electrolyser","SMR":"SMR",
                       "eSMR":"eSMR",
                "Gasification":"Gasification","Gasification_noWGR":"Gasification","Pyrolysis":"Pyrolysis",
           "Naphta_cracking":"Naphta cracking","Naphta_cracking_electrified":"Naphta cracking"}
tech_name_list={"Electrolyser":["Electrolyser"],"SMR":["SMR","eSMR"],
                "Gasification":["Gasification","Gasification_noWGR"],"Pyrolysis":["Pyrolysis"],
           "Naphta cracking":["Naphta_cracking","Naphta_cracking_electrified"]}
hydrogen_techs_name_colors={"Electrolyser":"green","SMR":"black",
                "Gasification":"firebrick","Pyrolysis":"turquoise",
                       "Naphta cracking":"darkkhaki"}



fig =go.Figure()
tech_usage=tech_usage_flow[(tech_usage_flow.Resource.isin(["Hydrogen"]))]

hydrogen_tech_usage=tech_usage[tech_usage.Technology.isin(hydrogen_techs)]
for tech in tech_name_list.keys():
    u=hydrogen_tech_usage[hydrogen_tech_usage.Technology.isin(tech_name_list[tech])]
    u=u[["Year","Flow"]].groupby(by=["Year"]).sum().reset_index()
    fig.add_trace(go.Scatter(x=u.Year,y=u.Flow*1e-6,name=tech,line=dict(width=0,color=hydrogen_techs_name_colors[tech]),
                             fillcolor=hydrogen_techs_name_colors[tech],hoverinfo="x+y",mode='lines',stackgroup='one'))


energy_input=resource_imports[(resource_imports.Resource.isin(primary_resource_conversion.keys()))]
energy_input=energy_input.drop(["Sector","Country"],axis=1).groupby(by=["Resource","Year"]).sum().reset_index()

fig.add_trace(go.Scatter(x=energy_input[energy_input.Resource=="Hydrogen"].Year,y=energy_input[energy_input.Resource=="Hydrogen"].Flow/33.3,
                     name="Imports",
                     line=dict(width=0,color="grey"),
                         fillcolor="grey",mode='lines',stackgroup='one'))

fig.update_layout(title='Hydrogen production and imports in EU-5+1',  title_x=0.5,title_y=0.97,
                   yaxis_title='Mt of H<sub>2</sub>')
fig.update_layout(
    font=dict(
        family="Times New Roman",size=24)
    )
fig.update_annotations(font_size=24)
fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.06,
    xanchor="right",
    x=0.9,font = dict(size=20)
))
# fig.update_yaxes(range=[0,6.5])
# fig.add_annotation(dict(font=dict(color="black",size=28),
#                                         x=-0.05,
#                                         y=1.15,
#                                         showarrow=False,
#                                         text="(e)",
#                                         textangle=0,
#                                         xanchor='left',
#                                         xref="paper",
#                                         yref="paper"))
fig.update_layout( paper_bgcolor="rgba(0,0,0,0)", plot_bgcolor="rgba(0,0,0,0)")
fig.show()


In [169]:
fig =go.Figure()
energy_input=resource_tech_inflow[(resource_tech_inflow.Resource.isin(primary_resource_conversion.keys()))]
energy_input=energy_input[energy_input.Technology!="Electrolyser"]
energy_input=energy_input.drop(["Sector","Country"],axis=1).groupby(by=["Resource","Year"]).sum().reset_index()

    
resource="Electricity"
fig.add_trace(go.Scatter(x=energy_input[(energy_input.Resource==resource)].Year,
                         y=energy_input[(energy_input.Resource==resource)].Flow,
                         name='Other electricity consumptions',
                         line=dict(width=0,color="yellow"),
                             fillcolor="yellow",mode='lines',stackgroup='one'))


energy_input=resource_tech_inflow[(resource_tech_inflow.Resource.isin(primary_resource_conversion.keys()))]
energy_input=energy_input[energy_input.Technology=="Electrolyser"]
energy_input=energy_input.drop(["Sector","Country"],axis=1).groupby(by=["Resource","Year"]).sum().reset_index()


resource="Electricity"
fig.add_trace(go.Scatter(x=energy_input[(energy_input.Resource==resource)].Year,
                         y=energy_input[(energy_input.Resource==resource)].Flow,
                         name='Electrolysers',
                         line=dict(width=0,color="turquoise"),
                             fillcolor="turquoise",mode='lines',stackgroup='one'))
#     fig.update_xaxes(title_text="Year", row=1, col=i+1)
        
fig.update_layout(title='Electricity consumption in EU-5+1',title_x=0.5,title_y=0.97,
                   yaxis_title='Electricity consumption (TWh)')
fig.update_layout(
    font=dict(
        family="Times New Roman",size=24)
    )
fig.update_annotations(font_size=24)
fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.06,
    xanchor="right",
    x=0.75,font = dict(size=20)
))
# fig.update_yaxes(range=[0,500])

# fig.add_annotation(dict(font=dict(color="black",size=28),
#                                         x=-0.05,
#                                         y=1.15,
#                                         showarrow=False,
#                                         text="(f)",
#                                         textangle=0,
#                                         xanchor='left',
#                                         xref="paper",
#                                         yref="paper"))

fig.update_layout( paper_bgcolor="rgba(0,0,0,0)", plot_bgcolor="rgba(0,0,0,0)")

fig.show()

# Technology results

In [170]:
tech_list=['BF-BOF',"DRI-EAF","Kiln-DRI-EAF","Electrowinning","EAF"]
steel_techs={"BF-BOF":[['BF-BOF'],['Coal','H2']],
             "BF-BOF w/ Biomass":[['BF-BOF'],['Bio-10%','Bio-30%','Bio-50%']],             
             "DRI-EAF-H2":[["DRI-EAF"],["H2"]],"DRI-EAF-CH4":[["DRI-EAF"],["CH4"]],
             "Electrowinning":[["Electrowinning"],[0,"50%_LF","25%_LF"]],"EAF":[["EAF"],[0]]}
steel_techs_colors={"BF-BOF":"Black","BF-BOF w/ Biomass":"brown",
                    "DRI-EAF-H2":"turquoise","DRI-EAF-CH4":'purple','DRI-EAF-Biomass':"firebrick",
                    "Electrowinning":"orange","EAF":"yellow"}


fig =go.Figure()
tech_usage=tech_usage_flow[tech_usage_flow.Resource.isin(["Steel_primary","Steel_secondary"])]
steel_tech_usage=tech_usage[tech_usage.Technology.isin(tech_list)]

# country="France"
# steel_tech_usage=steel_tech_usage[steel_tech_usage.Country==country]

for tech in steel_techs.keys():
        t_list=steel_techs[tech][0]
        tt_list=steel_techs[tech][1]
        u=steel_tech_usage[(steel_tech_usage.Technology.isin(t_list))&(steel_tech_usage.Tech_type.isin(tt_list))]
        u=u[["Year","Flow"]].groupby(by='Year').sum().reset_index()

        fig.add_trace(go.Scatter(x=u.Year,y=u.Flow*1e-6,name=tech,line=dict(width=0,color=steel_techs_colors[tech]), 
                             fillcolor=steel_techs_colors[tech],
                             hoverinfo="x+y",mode='lines',stackgroup='one'))

#     fig.update_xaxes(title_text="Year", row=1, col=i+1)
    
fig.update_layout(#title='Steel production per technology in EU-5+1',title_x=0.5,
                   yaxis_title='Production (Mt of steel)')
fig.update_layout(
    font=dict(
        family="Times New Roman",size=24)
    )
fig.update_annotations(font_size=24)
fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.1,
    xanchor="right",
    x=1.02,font = dict(size=18)
))
# fig.add_annotation(dict(font=dict(color="black",size=28),
#                                         x=-0.05,
#                                         y=1.15,
#                                         showarrow=False,
#                                         text="(a)",
#                                         textangle=0,
#                                         xanchor='left',
#                                         xref="paper",
#                                         yref="paper"))

fig.update_layout( paper_bgcolor="rgba(0,0,0,0)", plot_bgcolor="rgba(0,0,0,0)")
fig.show()

In [171]:
fig =go.Figure()
emissions_agg=emissions.drop(["Country"],axis=1).groupby(by=["Year","Sector"]).sum().reset_index()
captured_emissions_agg=captured_emissions.drop(["Technology","Country"],axis=1).groupby(by=["Year","Sector"]).sum().reset_index()
emissions_agg=emissions_agg[emissions_agg.Sector=="Steel"].drop("Sector",axis=1)
captured_emissions_agg=captured_emissions_agg[captured_emissions_agg.Sector=="Steel"].drop("Sector",axis=1)

fig.add_trace(go.Scatter(
    name="Emissions",
    x=emissions_agg.Year, y=emissions_agg.Emissions*1e-6,
    mode='lines',
    line=dict(width=0, color='grey'),
    stackgroup='one'))
fig.add_trace(go.Scatter(
    name="Captured emissions",
    x=captured_emissions_agg.Year, y=captured_emissions_agg.Captured_emissions*1e-6,
    mode='lines',
    line=dict(width=0, color='black'),
    stackgroup='one'))

#     fig.update_xaxes(title_text="Year", row=1, col=i+1)

fig.update_layout(#title='CO<sub>2</sub> emissions for steel production in EU-5+1',title_x=0.5,title_y=0.97,
                   yaxis_title='Emissions for steel (MtCO<sub>2</sub>)')
# fig.update_yaxes(range=[0,350])
fig.update_layout(
    font=dict(
        family="Times New Roman",size=24)
    )
fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.06,
    xanchor="right",
    x=0.75
))
fig.update_annotations(font_size=24)
# fig.add_annotation(dict(font=dict(color="black",size=28),
#                                         x=-0.05,
#                                         y=1.15,
#                                         showarrow=False,
#                                         text="(a)",
#                                         textangle=0,
#                                         xanchor='left',
#                                         xref="paper",
#                                         yref="paper"))
fig.update_layout( paper_bgcolor="rgba(0,0,0,0)", plot_bgcolor="rgba(0,0,0,0)")
fig.show()

In [172]:
olefins_techs=["Naphta_cracking","Naphta_cracking_electrified","MethanolToOlefins", "OxidativeCouplingOfMethane"]
olefins_techs_colors={"Naphta_cracking":"black","Naphta_cracking_electrified":"grey","MethanolToOlefins":"yellow", "OxidativeCouplingOfMethane":"purple"}
olefins_tech_name={"Naphta cracking":["Naphta_cracking"],"Naphta cracking electrified":["Naphta_cracking_electrified"],
                        "Methanol to Olefins (MTO)":["MethanolToOlefins"],
                        "Oxidative Coupling of Methane (OCM)":["OxidativeCouplingOfMethane"]}
olefins_techs_name_colors={"Naphta cracking":"black","Naphta cracking electrified":"grey",
                           "Methanol to Olefins (MTO)":"yellow", "Oxidative Coupling of Methane (OCM)":"purple"}



scennames=["Reference"]

fig =go.Figure()
tech_usage=tech_usage_flow[(tech_usage_flow.Resource.isin(["Olefins"]))]
olefins_tech_usage=tech_usage[tech_usage.Technology.isin(olefins_techs)]
for tech in olefins_tech_name.keys():
    u=olefins_tech_usage[olefins_tech_usage.Technology.isin(olefins_tech_name[tech])]
    u=u[["Year","Flow"]].groupby(by=["Year"]).sum().reset_index()
    fig.add_trace(go.Scatter(x=u.Year,y=u.Flow*1e-6,name=tech,line=dict(width=0,color=olefins_techs_name_colors[tech]),
                             fillcolor=olefins_techs_name_colors[tech],hoverinfo="x+y",mode='lines',stackgroup='one'))
#         fig.update_xaxes(title_text="Year", row=1, col=i+1)

fig.update_layout(title='Olefins production in EU-5+1',title_x=0.5,title_y=1,
                   yaxis_title='Production (Mt of olefins)')
fig.update_layout(
    font=dict(
        family="Times New Roman",size=24)
    )
fig.update_annotations(font_size=24)
fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=0.9,font = dict(size=20)
))
# fig.add_annotation(dict(font=dict(color="black",size=28),
#                                         x=-0.05,
#                                         y=1.15,
#                                         showarrow=False,
#                                         text="(c)",
#                                         textangle=0,
#                                         xanchor='left',
#                                         xref="paper",
#                                         yref="paper"))

fig.update_layout( paper_bgcolor="rgba(0,0,0,0)", plot_bgcolor="rgba(0,0,0,0)")
fig.show()

In [173]:
olefins_techs=["Naphta_cracking","Naphta_cracking_electrified","MethanolToOlefins", "OxidativeCouplingOfMethane"]
olefins_techs_colors={"Naphta_cracking":"black","Naphta_cracking_electrified":"grey","MethanolToOlefins":"yellow", "OxidativeCouplingOfMethane":"purple"}
olefins_tech_name={"Naphta cracking":["Naphta_cracking","Naphta_cracking_electrified"],
                        "Methanol to Olefins (MTO)":["MethanolToOlefins"],
                        "Oxidative Coupling of Methane (OCM)":["OxidativeCouplingOfMethane"]}
olefins_techs_name_colors={"Naphta cracking":"black","Methanol to Olefins (MTO)":"yellow", "Oxidative Coupling of Methane (OCM)":"purple"}

naphtha_techs=["Oil_Refining_for_Naphtha","BioNaphtha_making"]
naphtha_techs_colors={"Oil_Refining_for_Naphtha":"black","BioNaphtha_making":"green"}
naphtha_techs_name={"Oil_Refining_for_Naphtha":"Oil refining","BioNaphtha_making":"Bionaphtha"}
tech_usage=tech_usage_flow[(tech_usage_flow.Resource.isin(["Naphtha"]))]
naphtha_tech_usage=tech_usage[tech_usage.Technology.isin(naphtha_techs)]
naphtha_tech_usage=naphtha_tech_usage.groupby(["Technology","Year"]).sum().reset_index().pivot(index='Year', columns='Technology', values='Flow')
naphtha_tech_usage["BioNaphtha_making_distrib"]=naphtha_tech_usage["BioNaphtha_making"]/(naphtha_tech_usage["BioNaphtha_making"]+naphtha_tech_usage["Oil_Refining_for_Naphtha"])
naphtha_tech_usage["Oil_Refining_for_Naphtha_distrib"]=naphtha_tech_usage["Oil_Refining_for_Naphtha"]/(naphtha_tech_usage["BioNaphtha_making"]+naphtha_tech_usage["Oil_Refining_for_Naphtha"])

scennames=["Reference"]

fig =go.Figure()
tech_usage=tech_usage_flow[(tech_usage_flow.Resource.isin(["Olefins"]))]
olefins_tech_usage=tech_usage[tech_usage.Technology.isin(olefins_techs)]
for tech in olefins_tech_name.keys():
    if tech!="Naphta cracking":
        u=olefins_tech_usage[olefins_tech_usage.Technology.isin(olefins_tech_name[tech])]
        u=u[["Year","Flow"]].groupby(by=["Year"]).sum().reset_index()
        fig.add_trace(go.Scatter(x=u.Year,y=u.Flow*1e-6,name=tech,line=dict(width=0,color=olefins_techs_name_colors[tech]),
                                 fillcolor=olefins_techs_name_colors[tech],hoverinfo="x+y",mode='lines',stackgroup='one'))
    else:
        u=olefins_tech_usage[olefins_tech_usage.Technology.isin(olefins_tech_name[tech])]
        u=u[["Year","Flow"]].groupby(by=["Year"]).sum().reset_index()
        fig.add_trace(go.Scatter(x=u.Year,y=u.Flow*1e-6*naphtha_tech_usage["Oil_Refining_for_Naphtha_distrib"].to_numpy(),
                                 name="Naphtha cracking (Fossil)",line=dict(width=0,color="black"),
                                 fillcolor="black",hoverinfo="x+y",mode='lines',stackgroup='one'))
        fig.add_trace(go.Scatter(x=u.Year,y=u.Flow*1e-6*naphtha_tech_usage["BioNaphtha_making_distrib"].to_numpy(),
                                 name="Naphtha cracking (Biomass)",line=dict(width=0,color="green"),
                                 fillcolor="green",hoverinfo="x+y",mode='lines',stackgroup='one'))


fig.update_layout(title='Olefins production in EU-5+1',title_x=0.5,title_y=1,
                   yaxis_title='Production (Mt of olefins)')
fig.update_layout(
    font=dict(
        family="Times New Roman",size=24)
    )
fig.update_annotations(font_size=24)
fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=0.9,font = dict(size=20)
))
# fig.add_annotation(dict(font=dict(color="black",size=28),
#                                         x=-0.05,
#                                         y=1.15,
#                                         showarrow=False,
#                                         text="(c)",
#                                         textangle=0,
#                                         xanchor='left',
#                                         xref="paper",
#                                         yref="paper"))

fig.update_layout( paper_bgcolor="rgba(0,0,0,0)", plot_bgcolor="rgba(0,0,0,0)")
fig.show()

In [174]:
naphtha_techs=["Oil_Refining_for_Naphtha","BioNaphtha_making"]
naphtha_techs_colors={"Oil_Refining_for_Naphtha":"black","BioNaphtha_making":"green"}
naphtha_techs_name={"Oil_Refining_for_Naphtha":"Oil refining","BioNaphtha_making":"Bionaphtha"}



scennames=["Reference"]

fig =go.Figure()
tech_usage=tech_usage_flow[(tech_usage_flow.Resource.isin(["Naphtha"]))]
naphtha_tech_usage=tech_usage[tech_usage.Technology.isin(naphtha_techs)]
for tech in naphtha_techs:
    u=naphtha_tech_usage[naphtha_tech_usage.Technology==tech]
    u=u[["Year","Flow"]].groupby(by=["Year"]).sum().reset_index()
    fig.add_trace(go.Scatter(x=u.Year,y=u.Flow*1e-6,name=naphtha_techs_name[tech],line=dict(width=0,color=naphtha_techs_colors[tech]),
                             fillcolor=naphtha_techs_colors[tech],hoverinfo="x+y",mode='lines',stackgroup='one'))
#         fig.update_xaxes(title_text="Year", row=1, col=i+1)

fig.update_layout(title='Naphtha production in EU-5+1',title_x=0.5,title_y=1,
                   yaxis_title='Production (Mt of naphtha)')
fig.update_layout(
    font=dict(
        family="Times New Roman",size=24)
    )
fig.update_annotations(font_size=24)
fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.1,
    xanchor="right",
    x=0.65,font = dict(size=20)
))
# fig.add_annotation(dict(font=dict(color="black",size=28),
#                                         x=-0.05,
#                                         y=1.15,
#                                         showarrow=False,
#                                         text="(b)",
#                                         textangle=0,
#                                         xanchor='left',
#                                         xref="paper",
#                                         yref="paper"))

fig.update_layout( paper_bgcolor="rgba(0,0,0,0)", plot_bgcolor="rgba(0,0,0,0)")
fig.show()


In [175]:
fig =go.Figure()
emissions_agg=emissions.drop(["Country"],axis=1).groupby(by=["Year","Sector"]).sum().reset_index()
captured_emissions_agg=captured_emissions.drop(["Technology","Country"],axis=1).groupby(by=["Year","Sector"]).sum().reset_index()
emissions_agg=emissions_agg[emissions_agg.Sector=="Chemistry"].drop("Sector",axis=1)
captured_emissions_agg=captured_emissions_agg[captured_emissions_agg.Sector=="Chemistry"].drop("Sector",axis=1)

fig.add_trace(go.Scatter(
    name="Emissions",
    x=emissions_agg.Year, y=emissions_agg.Emissions*1e-6,
    mode='lines',
    line=dict(width=0, color='grey'),
    stackgroup='one'))
fig.add_trace(go.Scatter(
    name="Captured emissions",
    x=captured_emissions_agg.Year, y=captured_emissions_agg.Captured_emissions*1e-6,
    mode='lines',
    line=dict(width=0, color='black'),
    stackgroup='one'))

#     fig.update_xaxes(title_text="Year", row=1, col=i+1)

fig.update_layout(#title='CO<sub>2</sub> emissions for steel production in EU-5+1',title_x=0.5,title_y=0.97,
                   yaxis_title='Emissions for chemistry (MtCO<sub>2</sub>)')
# fig.update_yaxes(range=[0,350])
fig.update_layout(
    font=dict(
        family="Times New Roman",size=24)
    )
fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.06,
    xanchor="right",
    x=0.75
))
fig.update_annotations(font_size=24)
# fig.add_annotation(dict(font=dict(color="black",size=28),
#                                         x=-0.05,
#                                         y=1.15,
#                                         showarrow=False,
#                                         text="(a)",
#                                         textangle=0,
#                                         xanchor='left',
#                                         xref="paper",
#                                         yref="paper"))
fig.update_layout( paper_bgcolor="rgba(0,0,0,0)", plot_bgcolor="rgba(0,0,0,0)")
fig.show()

In [176]:
clinker_techs=["Semi_wet_pathway_clinker","Dry_without_precalciner_pathway_clinker","Dry_with_precalciner_pathway_clinker",
              "Electrified_dry_without_precalciner_pathway_clinker",
               "Electrified_dry_with_precalciner_pathway_clinker"
              ]
clinker_techs_colors={"Semi_wet_pathway_clinker":"darkgoldenrod",
                      "Dry_without_precalciner_pathway_clinker":"cornflowerblue",
                      "Dry_with_precalciner_pathway_clinker":"mediumblue",
              "Electrified_dry_without_precalciner_pathway_clinker":"yellowgreen",
               "Electrified_dry_with_precalciner_pathway_clinker":"yellow"
                     }

clinker_techs_name={"Semi_wet_pathway_clinker":"Semi-wet",
                      "Dry_without_precalciner_pathway_clinker":"Dry without precalciner",
                      "Dry_with_precalciner_pathway_clinker":"Dry with precalciner",
              "Electrified_dry_without_precalciner_pathway_clinker":"Electrified - Dry w/o precal.",
               "Electrified_dry_with_precalciner_pathway_clinker":"Electrified - Dry w/ precal."
                     }

scennames=["Reference"]

fig =go.Figure()
tech_usage=tech_usage_flow[(tech_usage_flow.Resource.isin(["Clinker"]))]
clinker_tech_usage=tech_usage[tech_usage.Technology.isin(clinker_techs)]
for tech in clinker_techs:
    u=clinker_tech_usage[clinker_tech_usage.Technology==tech]
    u=u[["Year","Flow"]].groupby(by=["Year"]).sum().reset_index()

    fig.add_trace(go.Scatter(x=u.Year,y=u.Flow*1e-6,name=clinker_techs_name[tech],line=dict(width=0,color=clinker_techs_colors[tech]),fillcolor=clinker_techs_colors[tech],
                             hoverinfo="x+y",mode='lines',stackgroup='one'))
#         fig.update_xaxes(title_text="Year", row=1, col=i+1)


fig.update_layout(#title='Clinker production per technology in EU-5+1', title_x=0.5,
#                    xaxis_title='Year',
                   yaxis_title='Production (Mt of clinker)')
fig.update_layout(
    font=dict(
        family="Times New Roman",size=24)
    )
fig.update_annotations(font_size=24)
fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.1,
    xanchor="right",
    x=0.85,font = dict(size=20)
))
fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.1,
    xanchor="right",
    x=0.8,#font = dict(size=16)
))

fig.update_layout( paper_bgcolor="rgba(0,0,0,0)", plot_bgcolor="rgba(0,0,0,0)")
fig.show()

In [177]:
country="France"
clinker_techs=["Semi_wet_pathway_clinker","Dry_without_precalciner_pathway_clinker","Dry_with_precalciner_pathway_clinker",
              "Electrified_dry_without_precalciner_pathway_clinker",
               "Electrified_dry_with_precalciner_pathway_clinker","Waste_kiln","Biomass_kiln","Coal_kiln","Gas_kiln","H2_kiln"
              ]

scennames=["Reference"]

fig =go.Figure()

u=resource_tech_inflow[
    resource_tech_inflow.Technology.isin(clinker_techs)].drop(["Technology","Tech_type"],axis=1).groupby(["Sector","Country","Resource","Year"]).sum().reset_index()
energy_input=u[(u.Resource.isin(primary_resource_conversion.keys()))]
energy_input=energy_input.drop(["Sector"],axis=1).groupby(by=["Resource","Year"]).sum().reset_index()
# energy_input=energy_input[energy_input.Country==country]
for resource in ["Waste","Biomass","Coal","Gas","Hydrogen","Electricity"]:
    if resource not in ["Olefins","Ammonia","Naphtha","CO","MeOH","Oil"]:
        fig.add_trace(go.Scatter(x=energy_input[energy_input.Resource==resource].Year,y=energy_input[energy_input.Resource==resource].Flow,name=resource,
                         line=dict(width=0,color=primary_resource_color[resource]),
                             fillcolor=primary_resource_color[resource],mode='lines',stackgroup='one'))
#             fig.add_trace(go.Scatter(x=energy_output[energy_output.Resource==resource].Year,y=energy_output[energy_output.Resource==resource].Flow,name=resource,
#                              line=dict(width=0,color=primary_resource_color[resource]),
#                                  fillcolor=primary_resource_color[resource],mode='lines',stackgroup='two',showlegend=False)
#                           ,row=1, col=i+1)

#     fig.update_xaxes(title_text="Year", row=1, col=i+1)
    
fig.update_layout(#title='Energy flows in EU-5+1', title_x=0.5,
                   yaxis_title='Energy consumption for clinker (TWh)')
# fig.update_yaxes(range=[-600,1500])
fig.update_layout(
    font=dict(
        family="Times New Roman",size=24)
    )
fig.update_annotations(font_size=24)
fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.1,
    xanchor="right",
    x=0.85,font = dict(size=20)
))

# fig.add_annotation(dict(font=dict(color="black",size=28),
#                                         x=-0.05,
#                                         y=1.15,
#                                         showarrow=False,
#                                         text="(d)",
#                                         textangle=0,
#                                         xanchor='left',
#                                         xref="paper",
#                                         yref="paper"))

fig.update_layout( paper_bgcolor="rgba(0,0,0,0)", plot_bgcolor="rgba(0,0,0,0)")
fig.show()

In [178]:
cement_types_color={"CEM_I":"black","CEM_II":"grey","CEM_III_VI":"yellow","LC3":"brown","Fast_Carb":"green","Solidia_Cement":"darkcyan","BYF_Cement":"springgreen"}
cement_types_name={"CEM_I":"CEM I","CEM_II":"CEM II","CEM_III_VI":"CEM III-VI","LC3":"LC3","Fast_Carb":"FastCarb (recycling)","Solidia_Cement":"Solidia","BYF_Cement":"BYF"}
cement_inflow=resource_inflow[(resource_inflow.Resource.isin(cement_types_color.keys()))]
cement_inflow=cement_inflow[["Year","Resource","Flow"]].groupby(by=["Year","Resource"]).sum().reset_index()
cement_outflow=resource_outflow[(resource_outflow.Resource.isin(cement_types_color.keys()))]
cement_outflow=cement_outflow[["Year","Resource","Flow"]].groupby(by=["Year","Resource"]).sum().reset_index()
clinker_prod=resource_outflow[(resource_outflow.Resource=="Clinker") & (resource_outflow.Sector=="Cement")]
clinker_prod=clinker_prod[["Year","Flow"]].groupby(by="Year").sum().reset_index()
df_merged=pd.concat([cement_outflow],ignore_index=True)
fig = go.Figure()

for resource in cement_types_color.keys():
    fig.add_trace(go.Scatter(x=df_merged[df_merged.Resource==resource].Year,y=df_merged[df_merged.Resource==resource].Flow*1e-6,name=cement_types_name[resource],
                         line=dict(width=0,color=cement_types_color[resource]),fillcolor=cement_types_color[resource]
                             ,hoverinfo="x+y",mode='lines',stackgroup='one'))
fig.update_layout(title='Cement production in EU-5+1',title_x=0.5,
#                    xaxis_title='Year',
                   yaxis_title='Production (Mt)')
fig.add_trace(go.Scatter(x=clinker_prod.Year, y=clinker_prod.Flow*1e-6,
                    mode='lines+markers',
                    name='Clinker',line=dict(color='black', width=3)))
fig.update_layout(legend=dict(
    yanchor="top",
    y=1.15,
    xanchor="left",
    x=-0.01,font = dict(size=20)
))
fig.update_layout(
    font=dict(
        family="Times New Roman",
        size=24,
    ),
    legend=dict(
                orientation="h")
                
)
fig.update_xaxes(range=[2015, 2050])
# fig.add_annotation(dict(font=dict(color="black",size=28),
#                                         x=-0.05,
#                                         y=1.15,
#                                         showarrow=False,
#                                         text="(c)",
#                                         textangle=0,
#                                         xanchor='left',
#                                         xref="paper",
#                                         yref="paper"))

fig.update_layout( paper_bgcolor="rgba(0,0,0,0)", plot_bgcolor="rgba(0,0,0,0)")
fig.update_layout(legend_traceorder="reversed")
fig.show()

In [179]:
fig =go.Figure()
emissions_agg=emissions.drop(["Country"],axis=1).groupby(by=["Year","Sector"]).sum().reset_index()
captured_emissions_agg=captured_emissions.drop(["Technology","Country"],axis=1).groupby(by=["Year","Sector"]).sum().reset_index()
emissions_agg=emissions_agg[emissions_agg.Sector=="Cement"].drop("Sector",axis=1)
captured_emissions_agg=captured_emissions_agg[captured_emissions_agg.Sector=="Cement"].drop("Sector",axis=1)

fig.add_trace(go.Scatter(
    name="Emissions",
    x=emissions_agg.Year, y=emissions_agg.Emissions*1e-6,
    mode='lines',
    line=dict(width=0, color='grey'),
    stackgroup='one'))
fig.add_trace(go.Scatter(
    name="Captured emissions",
    x=captured_emissions_agg.Year, y=captured_emissions_agg.Captured_emissions*1e-6,
    mode='lines',
    line=dict(width=0, color='black'),
    stackgroup='one'))

#     fig.update_xaxes(title_text="Year", row=1, col=i+1)

fig.update_layout(#title='CO<sub>2</sub> emissions for steel production in EU-5+1',title_x=0.5,title_y=0.97,
                   yaxis_title='Emissions for cement (MtCO<sub>2</sub>)')
# fig.update_yaxes(range=[0,350])
fig.update_layout(
    font=dict(
        family="Times New Roman",size=24)
    )
fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.06,
    xanchor="right",
    x=0.75
))
fig.update_annotations(font_size=24)
# fig.add_annotation(dict(font=dict(color="black",size=28),
#                                         x=-0.05,
#                                         y=1.15,
#                                         showarrow=False,
#                                         text="(a)",
#                                         textangle=0,
#                                         xanchor='left',
#                                         xref="paper",
#                                         yref="paper"))
fig.update_layout( paper_bgcolor="rgba(0,0,0,0)", plot_bgcolor="rgba(0,0,0,0)")
fig.show()

In [180]:
ammonia_techs=["Haber_Bosch","AmmoniaElectrolysis_SSAS","AmmoniaElectrolysis_2Step","MoltenSalt"]
ammonia_techs_colors={"Haber_Bosch":"brown","AmmoniaElectrolysis_SSAS":"yellow","AmmoniaElectrolysis_2Step":"orange","MoltenSalt":"grey"}

ammonia_techs_name={"Haber_Bosch":"Haber Bosch","AmmoniaElectrolysis_SSAS":"One Step Electrolysis",
                    "AmmoniaElectrolysis_2Step":"Two Steps Electrolysis","MoltenSalt":"Molten Salt"}

fig =go.Figure()
tech_usage=tech_usage_flow[(tech_usage_flow.Resource.isin(["Ammonia"]))]
ammonia_tech_usage=tech_usage[tech_usage.Technology.isin(ammonia_techs)]
for tech in ammonia_techs:
    u=ammonia_tech_usage[ammonia_tech_usage.Technology==tech]
    u=u[["Year","Flow"]].groupby(by=["Year"]).sum().reset_index()

    fig.add_trace(go.Scatter(x=u.Year,y=u.Flow*1e-6,name=ammonia_techs_name[tech],line=dict(width=0,color=ammonia_techs_colors[tech]),fillcolor=ammonia_techs_colors[tech],
                             hoverinfo="x+y",mode='lines',stackgroup='one'))
#         fig.update_xaxes(title_text="Year", row=1, col=i+1)


fig.update_layout(title='NH<sub>3</sub> production in EU-5+1', title_x=0.5,title_y=1,
#                    xaxis_title='Year',
                   yaxis_title='Production (Mt of ammonia)')
fig.update_layout(
    font=dict(
        family="Times New Roman",size=24)
    )
fig.update_annotations(font_size=24)
fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.1,
    xanchor="right",
    x=0.9,font = dict(size=19)
))

# fig.add_annotation(dict(font=dict(color="black",size=28),
#                                         x=-0.05,
#                                         y=1.15,
#                                         showarrow=False,
#                                         text="(e)",
#                                         textangle=0,
#                                         xanchor='left',
#                                         xref="paper",
#                                         yref="paper"))

fig.update_layout( paper_bgcolor="rgba(0,0,0,0)", plot_bgcolor="rgba(0,0,0,0)")

fig.show()

In [181]:
e_fuel_inflow=resource_inflow[(resource_inflow.Resource.isin(["E-Kerosene"]))]
e_fuel_inflow=e_fuel_inflow[["Year","Resource","Flow"]].groupby(by=["Year","Resource"]).sum().reset_index()
e_fuel_outflow=resource_outflow[(resource_outflow.Resource.isin(["E-Kerosene"]))]
e_fuel_outflow=e_fuel_outflow[["Year","Resource","Flow"]].groupby(by=["Year","Resource"]).sum().reset_index()
df_merged=pd.concat([e_fuel_outflow],ignore_index=True)
fig = go.Figure()


fig.add_trace(go.Scatter(x=df_merged[df_merged.Resource=="E-Kerosene"].Year,y=df_merged[df_merged.Resource=="E-Kerosene"].Flow*1e-6,
                     line=dict(width=0,color="grey"),fillcolor='grey'
                         ,hoverinfo="x+y",mode='lines',stackgroup='one'))
fig.update_layout(title='e-Kerosene production in EU-5+1',title_x=0.5,
#                    xaxis_title='Year',
                   yaxis_title='Production (Mt)')

fig.update_layout(legend=dict(
    yanchor="top",
    y=1.15,
    xanchor="left",
    x=0.35
))
fig.update_layout(
    font=dict(
        family="Times New Roman",
        size=26,
    ),
    legend=dict(
                orientation="h")
                
)
# fig.update_xaxes(range=[2015, 2050])
# fig.add_annotation(dict(font=dict(color="black",size=28),
#                                         x=-0.05,
#                                         y=1.15,
#                                         showarrow=False,
#                                         text="(c)",
#                                         textangle=0,
#                                         xanchor='left',
#                                         xref="paper",
#                                         yref="paper"))

fig.update_layout( paper_bgcolor="rgba(0,0,0,0)", plot_bgcolor="rgba(0,0,0,0)")
fig.show()

In [182]:
fig =go.Figure()
emissions_agg=emissions.drop(["Country"],axis=1).groupby(by=["Year","Sector"]).sum().reset_index()
captured_emissions_agg=captured_emissions.drop(["Technology","Country"],axis=1).groupby(by=["Year","Sector"]).sum().reset_index()
emissions_agg=emissions_agg[emissions_agg.Sector=="E-Fuels"].drop("Sector",axis=1)
captured_emissions_agg=captured_emissions_agg[captured_emissions_agg.Sector=="E-Fuels"].drop("Sector",axis=1)

fig.add_trace(go.Scatter(
    name="Emissions",
    x=emissions_agg.Year, y=emissions_agg.Emissions*1e-6,
    mode='lines',
    line=dict(width=0, color='grey'),
    stackgroup='one'))
fig.add_trace(go.Scatter(
    name="Captured emissions",
    x=captured_emissions_agg.Year, y=captured_emissions_agg.Captured_emissions*1e-6,
    mode='lines',
    line=dict(width=0, color='black'),
    stackgroup='one'))

#     fig.update_xaxes(title_text="Year", row=1, col=i+1)

fig.update_layout(#title='CO<sub>2</sub> emissions for steel production in EU-5+1',title_x=0.5,title_y=0.97,
                   yaxis_title='Emissions for E-Fuels (MtCO<sub>2</sub>)')
# fig.update_yaxes(range=[0,350])
fig.update_layout(
    font=dict(
        family="Times New Roman",size=24)
    )
fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.06,
    xanchor="right",
    x=0.75
))
fig.update_annotations(font_size=24)
# fig.add_annotation(dict(font=dict(color="black",size=28),
#                                         x=-0.05,
#                                         y=1.15,
#                                         showarrow=False,
#                                         text="(a)",
#                                         textangle=0,
#                                         xanchor='left',
#                                         xref="paper",
#                                         yref="paper"))
fig.update_layout( paper_bgcolor="rgba(0,0,0,0)", plot_bgcolor="rgba(0,0,0,0)")
fig.show()

In [183]:
meoh_techs=['Gas_ConvMeOHReactor_GasSteam','Gas_ConvMeOHReactor_ElecSteam','CO2_to_MeOH_3_Step_Gas','CO2_to_MeOH_3_Step_Elec',
            'CO2_to_MeOH_2_Step_Gas','CO2_to_MeOH_2_Step_Elec','CO2_to_MeOH_1_Step']
meoh_techs_name={"Conventional reactor":['Gas_ConvMeOHReactor_GasSteam','Gas_ConvMeOHReactor_ElecSteam'],
                "CO2 to MeOH - 1 step":['CO2_to_MeOH_1_Step'],
                "CO2 to MeOH - 2 step":['CO2_to_MeOH_2_Step_Gas','CO2_to_MeOH_2_Step_Elec'],
                "CO2 to MeOH - 3 step":['CO2_to_MeOH_3_Step_Gas','CO2_to_MeOH_3_Step_Elec']
                        
               }
meoh_techs_name_colors={"Conventional reactor":"yellow","CO2 to MeOH - 1 step":"firebrick","CO2 to MeOH - 2 step":"beige","CO2 to MeOH - 3 step":"orange"}



scennames=["Reference"]

fig =go.Figure()
tech_usage=tech_usage_flow[(tech_usage_flow.Resource.isin(["MeOH"]))]
meoh_tech_usage=tech_usage[tech_usage.Technology.isin(meoh_techs)]
for tech in meoh_techs_name.keys():
    u=meoh_tech_usage[meoh_tech_usage.Technology.isin(meoh_techs_name[tech])]
    u=u[["Year","Flow"]].groupby(by=["Year"]).sum().reset_index()

    fig.add_trace(go.Scatter(x=u.Year,y=u.Flow*1e-6,name=tech,line=dict(width=0,color=meoh_techs_name_colors[tech]),fillcolor=meoh_techs_name_colors[tech],
                             hoverinfo="x+y",mode='lines',stackgroup='one'))
#         fig.update_xaxes(title_text="Year", row=1, col=i+1)


fig.update_layout(title='MeOH production in EU-5+1', title_x=0.5,title_y=1,
#                    xaxis_title='Year',
                   yaxis_title='Production (Mt of MeOH)')
fig.update_layout(
    font=dict(
        family="Times New Roman",size=24)
    )
fig.update_annotations(font_size=24)
fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.1,
    xanchor="right",
    x=1,font = dict(size=18)
))

# fig.update_yaxes(range=[0,45])

# fig.add_annotation(dict(font=dict(color="black",size=28),
#                                         x=-0.05,
#                                         y=1.15,
#                                         showarrow=False,
#                                         text="(f)",
#                                         textangle=0,
#                                         xanchor='left',
#                                         xref="paper",
#                                         yref="paper"))

fig.update_layout( paper_bgcolor="rgba(0,0,0,0)", plot_bgcolor="rgba(0,0,0,0)")
fig.show()

In [184]:
glass_types_color={"Container_glass":"blue","Flat_glass":"green"}
glass_types_name={"Container_glass":"Container","Flat_glass":"Flat"}
glass_inflow=resource_inflow[(resource_inflow.Resource.isin(glass_types_color.keys()))]
glass_inflow=glass_inflow[["Year","Resource","Flow"]].groupby(by=["Year","Resource"]).sum().reset_index()
glass_outflow=resource_outflow[(resource_outflow.Resource.isin(glass_types_color.keys()))]
glass_outflow=glass_outflow[["Year","Resource","Flow"]].groupby(by=["Year","Resource"]).sum().reset_index()
df_merged=pd.concat([glass_outflow],ignore_index=True)
fig = go.Figure()

for resource in glass_types_color.keys():
    fig.add_trace(go.Scatter(x=df_merged[df_merged.Resource==resource].Year,y=df_merged[df_merged.Resource==resource].Flow*1e-6,name=glass_types_name[resource],
                         line=dict(width=0,color=glass_types_color[resource]),fillcolor=glass_types_color[resource]
                             ,hoverinfo="x+y",mode='lines',stackgroup='one'))
fig.update_layout(title='Glass production in EU-5+1',title_x=0.5,
#                    xaxis_title='Year',
                   yaxis_title='Production (Mt)')

fig.update_layout(legend=dict(
    yanchor="top",
    y=1.15,
    xanchor="left",
    x=0.35
))
fig.update_layout(
    font=dict(
        family="Times New Roman",
        size=26,
    ),
    legend=dict(
                orientation="h")
                
)
# fig.update_xaxes(range=[2015, 2050])
# fig.add_annotation(dict(font=dict(color="black",size=28),
#                                         x=-0.05,
#                                         y=1.15,
#                                         showarrow=False,
#                                         text="(c)",
#                                         textangle=0,
#                                         xanchor='left',
#                                         xref="paper",
#                                         yref="paper"))

fig.update_layout( paper_bgcolor="rgba(0,0,0,0)", plot_bgcolor="rgba(0,0,0,0)")
fig.show()

In [185]:
glass_techs=['Regenerative_end-port_container','Regenerative_side-port_container','Recuperative_container',
             'Oxy_fuel_container','TCR_container','Regenerative_end-port_flat','Regenerative_side-port_flat',
             'Recuperative_flat','Oxy_fuel_flat','TCR_flat',"Hybrid_melter_flat","Hybrid_melter_container",
             'All_electric_furnace_flat',"All_electric_furnace_container"]
glass_techs_name={"Regenerative":['Regenerative_end-port_container','Regenerative_side-port_container',
                                  'Regenerative_end-port_flat','Regenerative_side-port_flat'],
                "Recuperative":['Recuperative_container','Recuperative_flat'],
                "Oxy-fuel":['Oxy_fuel_container','TCR_container','Oxy_fuel_flat','TCR_flat'],
                "Electric":['All_electric_furnace_flat',"All_electric_furnace_container"],
                  "Hybrid":["Hybrid_melter_flat","Hybrid_melter_container"]
                        
               }
glass_techs_name_colors={"Regenerative":"blue","Recuperative":"orange","Oxy-fuel":"red","Electric":"yellow",
                         "Hybrid":"turquoise"}



scennames=["Reference"]

fig =go.Figure()
tech_usage=tech_usage_flow[(tech_usage_flow.Resource.isin(["Melted_container","Melted_flat"]))]
glass_tech_usage=tech_usage[tech_usage.Technology.isin(glass_techs)]
for tech in glass_techs_name.keys():
    u=glass_tech_usage[glass_tech_usage.Technology.isin(glass_techs_name[tech])]
    u=u[["Year","Flow"]].groupby(by=["Year"]).sum().reset_index()

    fig.add_trace(go.Scatter(x=u.Year,y=u.Flow*1e-6,name=tech,
                             line=dict(width=0,color=glass_techs_name_colors[tech]),
                             fillcolor=glass_techs_name_colors[tech],
                             hoverinfo="x+y",mode='lines',stackgroup='one'))
#         fig.update_xaxes(title_text="Year", row=1, col=i+1)


fig.update_layout(title='Glass production in EU-5+1', title_x=0.5,title_y=1,
#                    xaxis_title='Year',
                   yaxis_title='Production (Mt of glass)')
fig.update_layout(
    font=dict(
        family="Times New Roman",size=24)
    )
fig.update_annotations(font_size=24)
fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.1,
    xanchor="right",
    x=0.85,font = dict(size=18)
))

# fig.update_yaxes(range=[0,39])

# fig.add_annotation(dict(font=dict(color="black",size=28),
#                                         x=-0.05,
#                                         y=1.15,
#                                         showarrow=False,
#                                         text="(f)",
#                                         textangle=0,
#                                         xanchor='left',
#                                         xref="paper",
#                                         yref="paper"))

fig.update_layout( paper_bgcolor="rgba(0,0,0,0)", plot_bgcolor="rgba(0,0,0,0)")
fig.show()

In [186]:
fig =go.Figure()
emissions_agg=emissions.drop(["Country"],axis=1).groupby(by=["Year","Sector"]).sum().reset_index()
captured_emissions_agg=captured_emissions.drop(["Technology","Country"],axis=1).groupby(by=["Year","Sector"]).sum().reset_index()
emissions_agg=emissions_agg[emissions_agg.Sector=="Glass"].drop("Sector",axis=1)
captured_emissions_agg=captured_emissions_agg[captured_emissions_agg.Sector=="Glass"].drop("Sector",axis=1)

fig.add_trace(go.Scatter(
    name="Emissions",
    x=emissions_agg.Year, y=emissions_agg.Emissions*1e-6,
    mode='lines',
    line=dict(width=0, color='grey'),
    stackgroup='one'))
fig.add_trace(go.Scatter(
    name="Captured emissions",
    x=captured_emissions_agg.Year, y=captured_emissions_agg.Captured_emissions*1e-6,
    mode='lines',
    line=dict(width=0, color='black'),
    stackgroup='one'))

#     fig.update_xaxes(title_text="Year", row=1, col=i+1)

fig.update_layout(#title='CO<sub>2</sub> emissions for steel production in EU-5+1',title_x=0.5,title_y=0.97,
                   yaxis_title='Emissions for Glass (MtCO<sub>2</sub>)')
# fig.update_yaxes(range=[0,350])
fig.update_layout(
    font=dict(
        family="Times New Roman",size=24)
    )
fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.06,
    xanchor="right",
    x=0.75
))
fig.update_annotations(font_size=24)
# fig.add_annotation(dict(font=dict(color="black",size=28),
#                                         x=-0.05,
#                                         y=1.15,
#                                         showarrow=False,
#                                         text="(a)",
#                                         textangle=0,
#                                         xanchor='left',
#                                         xref="paper",
#                                         yref="paper"))
fig.update_layout( paper_bgcolor="rgba(0,0,0,0)", plot_bgcolor="rgba(0,0,0,0)")
fig.show()

In [187]:
country_colors={"France":"blue","Germany":"black","Italy":"green","Belgium":"yellow","Spain":"orange","Great Britain":"red"}
elec_power_conv={"Electricity":1,"Electricity_50%_LF":2,"Electricity_25%_LF":4}


fig =go.Figure()
consum=elec_consum_fc
consum=consum.drop("Sector",axis=1).groupby(by=["Resource","Country","Year"]).sum().reset_index()
for resource in ["Electricity","Electricity_50%_LF","Electricity_25%_LF"]:
    consum.loc[consum.Resource==resource,"Flow"]=elec_power_conv[resource]*consum.loc[consum.Resource==resource,"Flow"]*1e-6/8.76
consum=consum.drop("Resource",axis=1).groupby(by=["Country","Year"]).sum().reset_index() 
for country in country_colors.keys():
    fig.add_trace(go.Scatter(x=consum[(consum.Country==country)].Year,
                             y=consum[(consum.Country==country)].Flow,
                             name=country,
                             line=dict(width=0,color=country_colors[country]),
                                 fillcolor=country_colors[country],mode='lines',stackgroup='one'))
fig.update_layout(#title='ammonia production per technology in EU-5+1', title_x=0.5,
#                    xaxis_title='Year',
                   yaxis_title='Power requirement (GW)')
fig.update_layout(
    font=dict(
        family="Times New Roman",size=20)
    )
fig.update_annotations(font_size=20)
fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.1,
    xanchor="right",
    x=0.8,#font = dict(size=16)
))

fig.update_layout( paper_bgcolor="rgba(0,0,0,0)", plot_bgcolor="rgba(0,0,0,0)")
fig.show()

# Costs

In [188]:
#add an overall cost analysis
fig=go.Figure()
    
s_cost=sector_cost
s_cost=s_cost.drop(["Country","Sector"],axis=1).groupby(["Year"]).sum().reset_index()

c_cost=carbon_cost.drop(["Country","Sector"],axis=1).groupby(["Year"]).sum().reset_index()

t_cost=tech_cost.drop(["Country","Technology","Sector"],axis=1).groupby(["Year"]).sum().reset_index()

ccs_cpx_cost=ccs_capex_cost.drop(["Country","Sector"],axis=1).groupby(["Year"]).sum().reset_index()

ccs_opx_cost=ccs_opex_cost.drop(["Country","Sector"],axis=1).groupby(["Year"]).sum().reset_index()

opex_cost=s_cost.copy().set_index("Year")
opex_cost=opex_cost-c_cost.set_index("Year")-t_cost.set_index("Year")-ccs_cpx_cost.set_index("Year")
opex_cost=opex_cost.reset_index()
fig.add_trace(go.Scatter(x=t_cost.Year,
                         y=t_cost.Cost*1e-9,
                         name="CAPEX",
                         line=dict(width=0,color="blueviolet"),
                             fillcolor="blueviolet",mode='lines',stackgroup='one',showlegend=True))
fig.add_trace(go.Scatter(x=ccs_cpx_cost.Year,
                         y=ccs_cpx_cost.Cost*1e-9,
                         name="CCS-CAPEX",
                         line=dict(width=0,color="mediumorchid"),
                             fillcolor="mediumorchid",mode='lines',stackgroup='one',showlegend=True))
fig.add_trace(go.Scatter(x=c_cost.Year,
                         y=c_cost.Cost*1e-9,
                         name="Carbon cost",
                         line=dict(width=0,color="black"),
                             fillcolor="black",mode='lines',stackgroup='one',showlegend=True))

fig.add_trace(go.Scatter(x=opex_cost.Year,
                         y=opex_cost.Cost*1e-9,
                         name="OPEX",
                         line=dict(width=0,color="cornflowerblue"),
                             fillcolor="cornflowerblue",mode='lines',stackgroup='one',showlegend=True))
fig.update_yaxes(title_text="Annulized cost")

        
# fig.update_layout(
#     width=1000,
#     height=1000,#title_text="Annualised costs per sector and scenario in B€ in EU-5+1"
#                  )
fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.05,
    xanchor="right",
    x=0.8,font = dict(size=20)
))

fig.update_layout(
    font=dict(
        family="Times New Roman",size=24)
    )
fig.update_annotations(font_size=24)

fig.update_layout(yaxis=dict(ticksuffix="B€"))

fig.update_layout( paper_bgcolor="rgba(0,0,0,0)", plot_bgcolor="rgba(0,0,0,0)")
fig.show()
# fig.write_image("fig1.pdf",engine="kaleido")

In [189]:
fig =make_subplots(rows=5, cols=1,shared_yaxes=True,shared_xaxes=True,vertical_spacing = 0.05)
sect_list=["Steel","Chemistry","Cement","Glass","E-Fuels"]
showlegend=[False,False,True,False,False]
for j in range(len(sect_list)):
    sector=sect_list[j]
    s_cost=sector_cost
    s_cost=s_cost[s_cost.Sector==sector].drop(["Country","Sector"],axis=1).groupby(["Year"]).sum().reset_index()

    c_cost=carbon_cost[carbon_cost.Sector==sector].drop(["Country","Sector"],axis=1).groupby(["Year"]).sum().reset_index()

    t_cost=tech_cost[tech_cost.Sector==sector].drop(["Country","Technology","Sector"],axis=1).groupby(["Year"]).sum().reset_index()

    ccs_cpx_cost=ccs_capex_cost[ccs_capex_cost.Sector==sector].drop(["Country","Sector"],axis=1).groupby(["Year"]).sum().reset_index()

    ccs_opx_cost=ccs_opex_cost[ccs_opex_cost.Sector==sector].drop(["Country","Sector"],axis=1).groupby(["Year"]).sum().reset_index()

    opex_cost=s_cost.copy().set_index("Year")
    opex_cost=opex_cost-c_cost.set_index("Year")-t_cost.set_index("Year")-ccs_cpx_cost.set_index("Year")-ccs_opx_cost.set_index("Year")
    opex_cost=opex_cost.reset_index()
    fig.add_trace(go.Scatter(x=t_cost.Year,
                             y=t_cost.Cost*1e-9,
                             name="CAPEX",
                             line=dict(width=0,color="blueviolet"),
                                 fillcolor="blueviolet",mode='lines',stackgroup='one',showlegend=showlegend[j]),row=j+1,col=1)
    fig.add_trace(go.Scatter(x=ccs_cpx_cost.Year,
                             y=ccs_cpx_cost.Cost*1e-9,
                             name="CCS-CAPEX",
                             line=dict(width=0,color="mediumorchid"),
                                 fillcolor="mediumorchid",mode='lines',stackgroup='one',showlegend=showlegend[j]),row=j+1, col=1)
    fig.add_trace(go.Scatter(x=c_cost.Year,
                             y=c_cost.Cost*1e-9,
                             name="Carbon cost",
                             line=dict(width=0,color="black"),
                                 fillcolor="black",mode='lines',stackgroup='one',showlegend=showlegend[j]),row=j+1, col=1)

    fig.add_trace(go.Scatter(x=opex_cost.Year,
                             y=opex_cost.Cost*1e-9,
                             name="OPEX",
                             line=dict(width=0,color="cornflowerblue"),
                                 fillcolor="cornflowerblue",mode='lines',stackgroup='one',showlegend=showlegend[j]),row=j+1, col=1)
    fig.update_yaxes(title_text=sector, row=j+1, col=1)
    

        
fig.update_layout(width=1000,
    height=1000,#title_text="Annualised costs per sector and scenario in B€ in EU-5+1"
                 )
fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.05,
    xanchor="right",
    x=0.8,font = dict(size=20)
))

fig.update_layout(
    font=dict(
        family="Times New Roman",size=24)
    )
fig.update_annotations(font_size=24)

fig.update_layout(yaxis=dict(ticksuffix="B€"),
                 yaxis2=dict(ticksuffix="B€"),
                 yaxis3=dict(ticksuffix="B€"),
                yaxis4=dict(ticksuffix="B€"),
                  yaxis5=dict(ticksuffix="B€"),
                  
                 )

fig.update_layout( paper_bgcolor="rgba(0,0,0,0)", plot_bgcolor="rgba(0,0,0,0)")
fig.show()
# fig.write_image("fig1.pdf",engine="kaleido")

# Sankey

In [190]:
year=2050
year_inflow=resource_tech_inflow[(resource_tech_inflow.Year==year) & (resource_tech_inflow.Resource.isin(primary_resource_conversion.keys()))].drop(["Tech_type","Country","Year"],axis=1).rename(columns={"Flow":"In"}).groupby(by=["Sector","Technology","Resource"]).sum()
year_outflow=resource_tech_outflow[(resource_tech_outflow.Year==year)& (resource_tech_inflow.Resource.isin(primary_resource_conversion.keys()))].drop(["Tech_type","Country","Year"],axis=1).rename(columns={"Flow":"Out"}).groupby(by=["Sector","Technology","Resource"]).sum()

resource_index={}
rlist=list(primary_resource_color.keys())+["Steel","Cement","Biogas","Glass"]
colors={"Hydrogen":"lightblue","Gas":"violet","Coal":"dimgrey","Oil":"silver","Biomass":"lightsalmon",
        "Electricity":"palegoldenrod","MeOH":"darksalmon","Waste":"mediumseagreen","Olefins":"darkcyan",
        "Ammonia":"darksalmon","Naphtha":"wheat","Biogas":"lightgreen"}

for i in range(len(rlist)):
    resource_index[rlist[i]]=i

bond={"source":[],"target":[],"value":[],"label":[],"color":[]}

# rlist=list(primary_resource_color.keys())+["Biogas"]
# print(rlist)


tech_list=V['tech_tech_type_combinations_parameters'].keys()
hydrogen_techs=["Electrolyser","SMR",
                       "eSMR",
                "Gasification","Pyrolysis"]
olefins_techs=["Naphta_cracking","Naphta_cracking_electrified","MethanolToOlefins", "OxidativeCouplingOfMethane"]
naphtha_techs=["Oil_Refining_for_Naphtha","BioNaphtha_making"]
biogas_techs=["Biogas_Digester"]
synth_methane_techs=["Biogas_Methanation"]
nh3_techs=["Haber_Bosch","AmmoniaElectrolysis_SSAS","AmmoniaElectrolysis_2Step","MoltenSalt"]
meoh_techs=['Gas_ConvMeOHReactor_GasSteam','Gas_ConvMeOHReactor_ElecSteam','CO2_to_MeOH_3_Step_Gas','CO2_to_MeOH_3_Step_Elec','CO2_to_MeOH_2_Step_Gas','CO2_to_MeOH_2_Step_Elec','CO2_to_MeOH_1_Step']


for sector in ["Chemistry","Steel","Cement"]:
        for tech in tech_list:
            if tech in hydrogen_techs:
                for resource in rlist:
                    if tech not in ["Gasification"]:
                        try:
                            if abs(year_inflow.loc[(sector,tech,resource)].sum())>1e-3:
                                    bond['source'].append(resource_index[resource])
                                    bond["target"].append(resource_index["Hydrogen"])
                                    bond["value"].append(round(year_inflow.loc[(sector,tech,resource)].sum(),2))
                                    bond["label"].append(tech)
                                    bond["color"].append(colors[resource])
                        except:
                            pass



                    elif tech in ["Gasification"]:
                        try: #energy allocation
                            if abs(year_inflow.loc[(sector,tech,resource)].sum())>1e-3:
                                    bond['source'].append(resource_index[resource])
                                    bond["target"].append(resource_index["Hydrogen"])
                                    bond["value"].append(round(year_inflow.loc[(sector,tech,resource)].sum(),2)*2/3)
                                    bond["label"].append(tech)
                                    bond["color"].append(colors[resource])
                                    bond['source'].append(resource_index[resource])
                                    bond["target"].append(resource_index["Gas"])
                                    bond["value"].append(round(year_inflow.loc[(sector,tech,resource)].sum(),2)*1/3)
                                    bond["label"].append(tech)
                                    bond["color"].append(colors[resource])
                        except:
                            pass
                        
                        
                    else:
                        try:
                            if abs(year_inflow.loc[(sector,tech,resource)].sum())>1e-3:
                                bond['source'].append(resource_index[resource])
                                bond["target"].append(resource_index["Hydrogen"])
                                bond["value"].append(round(year_inflow.loc[(sector,tech,resource)].sum(),3))
                                bond["label"].append(tech)
                                bond["color"].append(colors[resource])
                        except:
                            pass
              
            
            elif tech in biogas_techs:
                try:
                    if abs(year_outflow.loc[(sector,tech,"Gas")].sum())>1e-3:
                        bond['source'].append(resource_index["Biogas"])
                        bond["target"].append(resource_index["Gas"])
                        bond["value"].append(-1*round(year_outflow.loc[(sector,tech,"Gas")].sum(),3))
                        bond["label"].append(tech)
                        bond["color"].append(colors["Biogas"])
                except:
                    pass
            elif tech in synth_methane_techs:
                for resource in rlist:
                    try:
                        if abs(year_outflow.loc[(sector,tech,"Gas")].sum())>1e-3:
                            bond['source'].append(resource_index[resource])
                            bond["target"].append(resource_index["Gas"])
                            bond["value"].append(round(year_inflow.loc[(sector,tech,resource)].sum(),3))
                            bond["label"].append(tech)
                            bond["color"].append(colors[resource])
                    except:
                        pass

            elif tech in olefins_techs:
                for resource in rlist:
                    if tech in ["Naphta_cracking"]:
                        try: #energy allocation
                                if abs(year_inflow.loc[(sector,tech,resource)].sum())>1e-3:
                                        bond['source'].append(resource_index[resource])
                                        bond["target"].append(resource_index["Hydrogen"])
                                        bond["value"].append(round(year_inflow.loc[(sector,tech,resource)].sum(),2)*0.06)
                                        bond["label"].append(tech)
                                        bond["color"].append(colors[resource])
                                        bond['source'].append(resource_index[resource])
                                        bond["target"].append(resource_index["Gas"])
                                        bond["value"].append(round(year_inflow.loc[(sector,tech,resource)].sum(),2)*0.03)
                                        bond["label"].append(tech)
                                        bond["color"].append(colors[resource])
                                        bond['source'].append(resource_index[resource])
                                        bond["target"].append(resource_index["Olefins"])
                                        bond["value"].append(round(year_inflow.loc[(sector,tech,resource)].sum(),2)*0.91)
                                        bond["label"].append(tech)
                                        bond["color"].append(colors[resource])
                        except:
                            pass  
                    elif tech in ["Naphta_cracking_electrified"]:
                        try: #energy allocation
                                if abs(year_inflow.loc[(sector,tech,resource)].sum())>1e-3:
                                        bond['source'].append(resource_index[resource])
                                        bond["target"].append(resource_index["Hydrogen"])
                                        bond["value"].append(round(year_inflow.loc[(sector,tech,resource)].sum(),2)*0.5)
                                        bond["label"].append(tech)
                                        bond["color"].append(colors[resource])
                                        bond['source'].append(resource_index[resource])
                                        bond["target"].append(resource_index["Gas"])
                                        bond["value"].append(round(year_inflow.loc[(sector,tech,resource)].sum(),2)*0.25)
                                        bond["label"].append(tech)
                                        bond["color"].append(colors[resource])
                                        bond['source'].append(resource_index[resource])
                                        bond["target"].append(resource_index["Olefins"])
                                        bond["value"].append(round(year_inflow.loc[(sector,tech,resource)].sum(),2)*0.7)
                                        bond["label"].append(tech)
                                        bond["color"].append(colors[resource])
                        except:
                            pass 


                    else:
                        try:
                            if abs(year_inflow.loc[(sector,tech,resource)].sum())>1e-3:
                                bond['source'].append(resource_index[resource])
                                bond["target"].append(resource_index["Olefins"])
                                bond["value"].append(round(year_inflow.loc[(sector,tech,resource)].sum(),3))
                                bond["label"].append(tech)
                                bond["color"].append(colors[resource])
                        except:
                            pass
            
            elif tech in naphtha_techs:
                for resource in rlist:
                    try:
                        if abs(year_inflow.loc[(sector,tech,resource)].sum())>1e-3:
                            bond['source'].append(resource_index[resource])
                            bond["target"].append(resource_index["Naphtha"])
                            bond["value"].append(round(year_inflow.loc[(sector,tech,resource)].sum(),3))
                            bond["label"].append(tech)
                            bond["color"].append(colors[resource])
                    except:
                        pass
            elif tech in nh3_techs:
                for resource in rlist:
                    try:
                        if abs(year_inflow.loc[(sector,tech,resource)].sum())>1e-3:
                            bond['source'].append(resource_index[resource])
                            bond["target"].append(resource_index["Ammonia"])
                            bond["value"].append(round(year_inflow.loc[(sector,tech,resource)].sum(),3))
                            bond["label"].append(tech)
                            bond["color"].append(colors[resource])
                    except:
                        pass
            elif tech in meoh_techs:
                for resource in rlist:
                    try:
                        if abs(year_inflow.loc[(sector,tech,resource)].sum())>1e-3:
                            bond['source'].append(resource_index[resource])
                            bond["target"].append(resource_index["MeOH"])
                            bond["value"].append(round(year_inflow.loc[(sector,tech,resource)].sum(),3))
                            bond["label"].append(tech)
                            bond["color"].append(colors[resource])
                    except:
                        pass
                
            else:
                if sector!="Chemistry":
                    for resource in rlist:
                        try:
                            if abs(year_inflow.loc[(sector,tech,resource)].sum())>1e-3:
                                bond['source'].append(resource_index[resource])
                                bond["target"].append(resource_index[sector])
                                bond["value"].append(round(year_inflow.loc[(sector,tech,resource)].sum(),2))
                                bond["label"].append(tech)
                                bond["color"].append(colors[resource])
                        except:
                            pass

#                         try:
#                             if abs(year_outflow.loc[(sector,tech,resource)].sum())>1e-3:
#                                 bond['source'].append(resource_index[sector])
#                                 bond["target"].append(resource_index[resource])
#                                 bond["value"].append(-1*round(year_outflow.loc[(sector,tech,resource)].sum(),3))
#                                 bond["label"].append(tech)
#                                 bond["color"].append(colors[resource])
#                         except:
#                             pass
    

                        
node_names=[]
for node in list(primary_resource_color.keys())+["Steel","Cement","Biogas","Glass"]:
    if node in ["Olefins","Steel","Cement","Glass"]:
        node_names.append(node)
    else:
        count=0
        for i in range(len(bond["source"])):
            if resource_index[node]==bond["source"][i]:
                count+=bond["value"][i]
        if round(count,2)<0.01:
            node_names.append(node)
        else:
            node_names.append(node+' - '+str(round(count,2))+'TWh')
           
        
            
            
fig = go.Figure(go.Sankey(valueformat = ".0f",
    valuesuffix = "TWh",
    node = {
        "label": node_names,
        "color": list(primary_resource_color.values())+["orange","orange","lawngreen","orange"], "pad":30,"thickness":30

    },

    link = bond))

fig.update_layout(title='Energy flows in EU-5+1 in '+str(year))

fig.show()

In [191]:
year=2050
bond={"source":[],"target":[],"value":[],"label":[],"color":[]}

direct_emissions_sector=V["V_emissions_tech_type_plus"].reset_index().rename(columns={"level_0":"Technology","level_1":"Tech_type","level_2":"Sector","level_3":"Country","level_4":"Year",0:"Emissions"}).sort_values(by="Year")[["Sector","Year","Emissions"]].groupby(["Sector","Year"]).sum().reset_index()
direct_emissions_sector=direct_emissions_sector[direct_emissions_sector.Year==year].drop("Year",axis=1).set_index(["Sector"])
consumed_emissions_sector=V["V_emissions_tech_type_minus"].reset_index().rename(columns={"level_0":"Technology","level_1":"Tech_type","level_2":"Sector","level_3":"Country","level_4":"Year",0:"Emissions"}).sort_values(by="Year")[["Sector","Year","Emissions"]].groupby(["Sector","Year"]).sum().reset_index()
consumed_emissions_sector=consumed_emissions_sector[consumed_emissions_sector.Year==year].drop("Year",axis=1).set_index(["Sector"])

# direct_emissions_sector=resource_outflow[(resource_outflow.Resource=="CO2")&(resource_outflow.Year==year)][["Sector","Flow"]].groupby(["Sector"]).sum()
# consumed_emissions_sector=resource_inflow[(resource_inflow.Resource=="CO2")&(resource_inflow.Year==year)][["Sector","Flow"]].groupby(["Sector"]).sum()

emissions_sector=emissions.groupby(["Sector","Year"]).sum().reset_index()
emissions_sector=emissions_sector[emissions_sector.Year==year].drop("Year",axis=1).set_index(["Sector"])
cc_sector=captured_emissions.drop("Technology",axis=1).groupby(["Sector","Year"]).sum().reset_index()
cc_sector=cc_sector[cc_sector.Year==year].drop("Year",axis=1).set_index(["Sector"])

waste_biogenic_emissions=-1*V["Parameters"]["RESOURCES_parameters"].loc[("Waste","Steel","France",2015),"emissions_r"]
biomass_biogenic_emissions=-1*V["Parameters"]["RESOURCES_parameters"].loc[("Biomass","Steel","France",2015),"emissions_r"]
r_imports=V["V_resource_imports"].reset_index().rename(columns={"level_0":"Resource","level_1":"Sector","level_2":"Country","level_3":"Year",0:"Flow"}).sort_values(by="Year").groupby(by=["Resource","Sector","Country","Year"]).sum().reset_index()
biogenic_carbon=r_imports[r_imports.Resource.isin(["Biomass","Waste"])].groupby(by=["Resource","Sector","Year"]).sum()
biogenic_carbon=biogenic_carbon.loc[(slice(None),slice(None),year)].reset_index().set_index(["Sector","Resource"])
biogas_neg_emissions=V["V_emissions_tech_type_minus"].reset_index().rename(columns={"level_0":"Technology","level_1":"Tech_type","level_2":"Sector","level_3":"Country","level_4":"Year",0:"Emissions"}).sort_values(by="Year")[["Technology","Sector","Year","Emissions"]].groupby(["Technology","Sector","Year"]).sum().reset_index()
biogas_neg_emissions=biogas_neg_emissions[(biogas_neg_emissions.Technology=="Biogas_Digester")&(biogas_neg_emissions.Year==year)].drop(["Year","Technology"],axis=1).set_index(["Sector"])



sources_index={}
source_list=["Direct emissions","Steel","Cement","Chemistry","Glass","E-Fuels","Capture","Emissions Stored","Biogenic carbon"]
sources_colors={"Direct emissions":"black","Steel":"rosybrown","Cement":"rosybrown",
                "Chemistry":"rosybrown","Glass":"rosybrown","E-Fuels":"rosybrown",
                "Capture":"black","Emissions Stored":"black","Biogenic carbon":"green"}

for i in range(len(source_list)):
    sources_index[source_list[i]]=i

stored_emissions=0
for sector in ["Steel","Cement","Chemistry","Glass","E-Fuels"]:
    bond['source'].append(sources_index[sector])
    bond["target"].append(sources_index["Direct emissions"])
    bond["value"].append(round((direct_emissions_sector.loc[(sector)].sum()-cc_sector.loc[(sector)].sum())*1e-6,3))
    bond["label"].append("")
    bond["color"].append("silver")
    
    bond['source'].append(sources_index["Capture"])
    bond["target"].append(sources_index[sector])
    bond["value"].append(round((biogas_neg_emissions.loc[(sector)].sum()-consumed_emissions_sector.loc[(sector)].sum())*1e-6,3))
    bond["label"].append("")
    bond["color"].append("aquamarine")
    stored_emissions+=(consumed_emissions_sector.loc[(sector)].sum()-biogas_neg_emissions.loc[(sector)].sum())*1e-6
    
    bond['source'].append(sources_index[sector])
    bond["target"].append(sources_index["Capture"])
    bond["value"].append(round(cc_sector.loc[(sector)].sum()*1e-6,3))
    bond["label"].append("")
    bond["color"].append("aquamarine")
    stored_emissions+=cc_sector.loc[(sector)].sum()*1e-6
    
    biomass_capture=biogenic_carbon.loc[(sector,"Biomass"),"Flow"].sum()*biomass_biogenic_emissions*1e-6
    bond['source'].append(sources_index["Direct emissions"])
    bond["target"].append(sources_index[sector])
    bond["value"].append(round(biomass_capture,3))
    bond["label"].append("")
    bond["color"].append("yellowgreen")
    
    waste_capture=biogenic_carbon.loc[(sector,"Waste"),"Flow"].sum()*waste_biogenic_emissions*1e-6
    bond['source'].append(sources_index["Direct emissions"])
    bond["target"].append(sources_index[sector])
    bond["value"].append(round(waste_capture,3))
    bond["label"].append("")
    bond["color"].append("yellowgreen")
    
    bond['source'].append(sources_index["Direct emissions"])
    bond["target"].append(sources_index[sector])
    bond["value"].append(round(biogas_neg_emissions.loc[(sector)].sum()*-1e-6,3))
    bond["label"].append("")
    bond["color"].append("yellowgreen")
    

bond['source'].append(sources_index["Capture"])
bond["target"].append(sources_index["Emissions Stored"])
bond["value"].append(round(stored_emissions,3))
bond["label"].append("")
bond["color"].append("cornflowerblue")


node_names=[]
for node in source_list:
    if node in ["Chemistry","Steel","Cement","Glass","E-Fuels"]:
        inflow=0
        outflow=0
        capture=0
        for i in range(len(bond["source"])):
            if sources_index[node]==bond["target"][i]:
                inflow+=bond["value"][i]
            elif sources_index[node]==bond["source"][i]:
                outflow+=bond["value"][i]
                if sources_index["Capture"]==bond["target"][i]:
                    capture+=bond["value"][i]
               
        if round(inflow,2)>0.01:
            if round(capture,2)>0.01:
                node_names.append(node+' <br>'+str(round(outflow,0))+'MtCO<sub>2</sub> (incl. '+str(round(capture,0))+'MtCO<sub>2</sub> captured) <br>-'\
                              +str(round(inflow,0))+'MtCO<sub>2</sub>')
            else:
                node_names.append(node+' <br>'+str(round(outflow,0))+'MtCO<sub>2</sub> <br>-'\
                              +str(round(inflow,0))+'MtCO<sub>2</sub>')
        else:
            node_names.append(node+' <br>'+str(round(outflow,0))+'MtCO<sub>2</sub>')
        
    elif node in ["Emissions Stored","Capture"]:
        inflow=0
        outflow=0
        for i in range(len(bond["source"])):
            if sources_index[node]==bond["target"][i]:
                inflow+=bond["value"][i]
            elif sources_index[node]==bond["source"][i]:
                outflow+=bond["value"][i]
                
        node_names.append(node+' <br>'+str(round(inflow,0))+'MtCO<sub>2</sub>')
        
    elif node in ["Direct emissions"]:
        inflow=0
        outflow=0
        for i in range(len(bond["source"])):
            if sources_index[node]==bond["target"][i]:
                inflow+=bond["value"][i]
            elif sources_index[node]==bond["source"][i]:
                outflow+=bond["value"][i]
        if round(outflow,2)>0.01:        
            node_names.append(node+' <br>'+str(round(inflow,0))+'MtCO<sub>2</sub> <br>-'\
                              +str(round(outflow,0))+'MtCO<sub>2</sub> biogenic')
        else:
            node_names.append(node+' <br>+'+str(round(inflow,0))+'MtCO<sub>2</sub>')
            
    
    else:
        node_names.append(node)
           
        
            
            
fig = go.Figure(go.Sankey(valueformat = ".0f",
    valuesuffix = "MtCO<sub>2</sub>",
    node = {
        "label": node_names,
        "color": list(sources_colors.values()), "pad":30,"thickness":30

    },visible=True, hoverinfo="all",

   link = bond))

fig.update_layout(title='Emission (scope 1) flows in EU-5+1 in '+str(year))


fig.show()